In [3]:
# Dashboard to display survey results, based on bokeh

In [4]:
# IDEA Build a german map HGF center celection plot.
# Is there some pic with HGF centers?

In [5]:
# TODO: maybe only read in results from an evaluation

Design choices: for plots data needs be in the form
```
{'All': array([  0,   0, 130, 128, 148, 272,   0]),
 'x' : 'careerLevel',
 'y' : 'researchArea'
 'careerLevel': ['Director (of the institute)',
  'Other',
  'PhD student',
  'Postdoc',
  'Principal Investigator',
  'Research associate',
  'Undergraduate / Masters student'],
 'researchArea': ['All', 'Engineering Science',
  'Physics',
  'Life Science',
  'Earth Science',
  'Chemistry',
  'Other',
  'Psychology',
  'Mathematics'],
 'Engineering Science': array([  0,   0,  47,  30,  52, 134,   0]),
 'Physics': array([ 0,  0, 33, 38, 39, 57,  0]),
 'Life Science': array([ 0,  0, 28, 29, 27, 33,  0]),
 'Earth Science': array([ 0,  0,  8, 11, 18, 32,  0]),
 'Chemistry': array([ 0,  0,  9, 12,  6,  4,  0]),
 'Other': array([0, 0, 1, 2, 3, 6, 0]),
 'Psychology': array([0, 0, 3, 2, 3, 2, 0]),
 'Mathematics': array([0, 0, 1, 4, 0, 4, 0])}
 ```
 

In [6]:
import os
import sys

from typing import List, Tuple
from pandas import read_csv
import pandas as pd
import random
import numpy as np
from bokeh.io import output_notebook, show, push_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Div, Select, Slider, TextInput
from bokeh.plotting import figure as bokeh_figure
from os.path import dirname, join
from bokeh.models import HelpTool, HoverTool
from bokeh.palettes import Category20c
from bokeh.transform import factor_cmap
from bokeh.transform import cumsum
from bokeh.models import FactorRange
from bokeh.transform import dodge
from bokeh.models import Panel, Tabs
from bokeh.layouts import column, row
from bokeh.models import Dropdown
from bokeh.palettes import RdYlBu3
from bokeh.palettes import Category20c

from bokeh.plotting import curdoc
from bokeh.models import ColumnDataSource, Div, Select
from bokeh.plotting import figure as bokeh_figure
from bokeh.models import CheckboxButtonGroup, CustomJS
from bokeh.models import MultiChoice
from bokeh.models import Paragraph
from masci_tools.vis.bokeh_plots import bokeh_scatter, bokeh_multi_scatter, bokeh_line

from bokeh.plotting import figure
import panel as pn

sys.path.append(".")


In [7]:
from analysis import filter_dataframe, calculate_crosstab
#from analysis import prepare_data_research_field


In [11]:
pn.extension()

In [8]:
output_notebook()

Loading BokehJS ...

In [9]:
def prepare_data_research_field(df: pd.DataFrame, key:str):
    """AI is creating summary for prepare_data_researchfield

    :param df: [description]
    :type df: pd.DataFrame
    :param key: [description]
    :type key: str
    """
    all_areas = df[key].value_counts()
    all_areas = all_areas.sort_index()
    data = {'All': {'counts': all_areas.values, 'values': list(all_areas.keys())}}
    research_areas = list(df['researchArea'].value_counts().keys())
    for area in research_areas:
        area_counts = df[df["researchArea"] == area][key].value_counts()
        area_counts = area_counts.sort_index()
        data[area] = {'counts': area_counts.values, 'values': list(area_counts.keys())}
    
    return data

In [8]:
# To embed a bokeh sever hosted somewhere else in a webpage one can add (https://docs.bokeh.org/en/latest/docs/user_guide/embed.html#app-documents):
from bokeh.embed import server_document
script = server_document("https://demo.bokeh.org/sliders")
script

'<script id="1002">\n  (function() {\n    const xhr = new XMLHttpRequest()\n    xhr.responseType = \'blob\';\n    xhr.open(\'GET\', "https://demo.bokeh.org/sliders/autoload.js?bokeh-autoload-element=1002&bokeh-app-path=/sliders&bokeh-absolute-url=https://demo.bokeh.org/sliders", true);\n    xhr.onload = function (event) {\n      const script = document.createElement(\'script\');\n      const src = URL.createObjectURL(event.target.response);\n      script.src = src;\n      document.body.appendChild(script);\n    };\n    xhr.send();\n  })();\n</script>'

In [9]:
pwd = os.getcwd()

In [10]:
datafilepath_raw = join(pwd, './data/HMC_community_survey2021_merged_all_replies.csv')
datafilepath= join(pwd, './data/20211130_HMCCommSurvey_clean.csv')

In [11]:
# read in data
surveydata = read_csv(datafilepath)

In [12]:
surveydata.describe()

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,dataAmount_lsf,dataOrigin_MeasVsSim,dataOrigin_SelvVsReuse,id,interviewtime,lastpage,pubAmount,researchAreaAGRI_other,researchAreaMATH_other,seed,Unnamed: 270,Unnamed: 271
count,1009.000000,1009.000000,1009.000000,174.000000,609.000000,609.000000,1009.000000,1009.000000,1009.000000,599.000000,0.0,0.0,1.009000e+03,0.0,0.0
mean,504.000000,504.000000,504.000000,37.442529,2.609195,2.320197,91.664024,771.469187,3.127849,25.617696,NaN,NaN,1.031139e+09,NaN,NaN
std,291.417513,291.417513,291.417513,43.806387,1.401942,1.311990,53.913650,2024.923251,2.169528,30.566596,NaN,NaN,6.130695e+08,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,-1.000000,0.000000,NaN,NaN,3.519180e+05,NaN,NaN
25%,252.000000,252.000000,252.000000,0.000000,1.000000,1.000000,46.000000,55.310000,1.000000,0.000000,NaN,NaN,5.256528e+08,NaN,NaN
50%,504.000000,504.000000,504.000000,5.000000,2.000000,2.000000,90.000000,423.360000,5.000000,10.000000,NaN,NaN,9.917112e+08,NaN,NaN
75%,756.000000,756.000000,756.000000,90.000000,4.000000,3.000000,133.000000,777.500000,5.000000,40.000000,NaN,NaN,1.546594e+09,NaN,NaN
max,1008.000000,1008.000000,1008.000000,100.000000,5.000000,5.000000,221.000000,25763.500000,5.000000,100.000000,NaN,NaN,2.145042e+09,NaN,NaN


In [40]:
questions = ['1', '2', '3']
FILTER_OPTIONS =['a', 'b', 'c']

In [99]:
question_select = Select(title="Question", value="careerLevel",
               options=questions)
filter_select = Select(title="Filter data", value="All",
               options=FILTER_OPTIONS)

multi_choice = MultiChoice(title="Filter data", value=["All", "Physics"], options=FILTER_OPTIONS)
multi_choice.js_on_change("value", CustomJS(code="""
    console.log('multi_choice: value=' + this.value, this.toString())
"""))

#LABELS = open(join(pwd, 'dashboard/data/filters.txt')).read().split('\n')

chart_select = Select(title="Chart type", value="bar",
               options=['bar', 'pie'])

lang_select = Select(title="Language", value="en",
               options=['en', 'de'])

# Controls 2
question_select2 = Select(title="Question X-Axis", value="careerLevel",
               options=questions)
question_select3 = Select(title="Question Y-Axis", value="docStructured",
               options=questions)

In [ ]:
question_select.description

In [15]:
list(surveydata.keys())

['Unnamed: 0.2',
 'Unnamed: 0',
 'Unnamed: 0.1',
 'careerLevel',
 'careerLevel_other',
 'centerAffiliation',
 'centerAffiliation_other',
 'dataAmount_lsf',
 'dataAnalDuration_sub',
 'dataFormats_1',
 'dataFormats_10',
 'dataFormats_11',
 'dataFormats_12',
 'dataFormats_13',
 'dataFormats_14',
 'dataFormats_15',
 'dataFormats_2',
 'dataFormats_3',
 'dataFormats_4',
 'dataFormats_5',
 'dataFormats_6',
 'dataFormats_7',
 'dataFormats_8',
 'dataFormats_9',
 'dataFormats_other',
 'dataGenMethodSpec_1_1',
 'dataGenMethodSpec_1_2',
 'dataGenMethodSpec_1_3',
 'dataGenMethodSpec_2_1',
 'dataGenMethodSpec_2_2',
 'dataGenMethodSpec_2_3',
 'dataGenMethodSpec_3_1',
 'dataGenMethodSpec_3_2',
 'dataGenMethodSpec_3_3',
 'dataGenMethodSpec_4_1',
 'dataGenMethodSpec_4_2',
 'dataGenMethodSpec_4_3',
 'dataGenMethodSpec_5_1',
 'dataGenMethodSpec_5_2',
 'dataGenMethodSpec_5_3',
 'dataGenMethodSpec_6_1',
 'dataGenMethodSpec_6_2',
 'dataGenMethodSpec_6_3',
 'dataGenMethodSpec_7_1',
 'dataGenMethodSpec_7_2',
 

In [16]:
surveydata['researchArea'].value_counts()

Engineering Science    306
Physics                194
Life Science           157
Earth Science           81
Chemistry               40
Other                   19
Psychology              13
Mathematics             11
Name: researchArea, dtype: int64

In [17]:
q1_key = 'careerLevel'
q2_key = 'docStructured'
calculate_crosstab(surveydata, q1_key, q2_key) 

,careerLevel,docStructured,value,total,percentage
0,Director (of the institute),I am not sure,1,22,4.545455
1,Other,I am not sure,2,65,3.076923
2,PhD student,I am not sure,20,130,15.384615
3,Postdoc,I am not sure,31,128,24.218750
4,Principal Investigator,I am not sure,19,148,12.837838
5,Research associate,I am not sure,27,272,9.926471
6,Undergraduate / Masters student,I am not sure,1,24,4.166667
7,Director (of the institute),No,6,22,27.272727
8,Other,No,13,65,20.000000
9,PhD student,No,32,130,24.615385


In [18]:
#pprint(list(surveydata.keys()))

In [19]:
to_exclude = ['Other', 'Undergraduate / Masters student', 'Director (of the institute)']
df = filter_dataframe(surveydata, include=["careerLevel", "researchArea"], exclude=[("careerLevel", to_exclude)])
counts = df.value_counts()
counts2 = df.value_counts()
counts

careerLevel             researchArea       
Research associate      Engineering Science    134
                        Physics                 57
Principal Investigator  Engineering Science     52
PhD student             Engineering Science     47
Principal Investigator  Physics                 39
Postdoc                 Physics                 38
PhD student             Physics                 33
Research associate      Life Science            33
                        Earth Science           32
Postdoc                 Engineering Science     30
                        Life Science            29
PhD student             Life Science            28
Principal Investigator  Life Science            27
                        Earth Science           18
Postdoc                 Chemistry               12
                        Earth Science           11
PhD student             Chemistry                9
                        Earth Science            8
Principal Investigator  Chemistry     

In [20]:
counts.keys()

MultiIndex([(    'Research associate', 'Engineering Science'),
            (    'Research associate',             'Physics'),
            ('Principal Investigator', 'Engineering Science'),
            (           'PhD student', 'Engineering Science'),
            ('Principal Investigator',             'Physics'),
            (               'Postdoc',             'Physics'),
            (           'PhD student',             'Physics'),
            (    'Research associate',        'Life Science'),
            (    'Research associate',       'Earth Science'),
            (               'Postdoc', 'Engineering Science'),
            (               'Postdoc',        'Life Science'),
            (           'PhD student',        'Life Science'),
            ('Principal Investigator',        'Life Science'),
            ('Principal Investigator',       'Earth Science'),
            (               'Postdoc',           'Chemistry'),
            (               'Postdoc',       'Earth Sci

In [21]:
list(surveydata['researchArea'].value_counts().keys())

['Engineering Science',
 'Physics',
 'Life Science',
 'Earth Science',
 'Chemistry',
 'Other',
 'Psychology',
 'Mathematics']

In [22]:
research_areas = list(surveydata['researchArea'].value_counts().keys())
research_areas

['Engineering Science',
 'Physics',
 'Life Science',
 'Earth Science',
 'Chemistry',
 'Other',
 'Psychology',
 'Mathematics']

In [23]:
key = "careerLevel"

df = filter_dataframe(surveydata, include=[key, "researchArea"], exclude=[(key, to_exclude)])
all_areas = df[key].value_counts()
all_areas = all_areas.sort_index()
data = prepare_data_research_field(df, key)

In [24]:
df.value_counts()

careerLevel             researchArea       
Research associate      Engineering Science    134
                        Physics                 57
Principal Investigator  Engineering Science     52
PhD student             Engineering Science     47
Principal Investigator  Physics                 39
Postdoc                 Physics                 38
PhD student             Physics                 33
Research associate      Life Science            33
                        Earth Science           32
Postdoc                 Engineering Science     30
                        Life Science            29
PhD student             Life Science            28
Principal Investigator  Life Science            27
                        Earth Science           18
Postdoc                 Chemistry               12
                        Earth Science           11
PhD student             Chemistry                9
                        Earth Science            8
Principal Investigator  Chemistry     

In [25]:
data

{'All': {'counts': array([  0,   0, 130, 128, 148, 272,   0]),
  'values': ['Director (of the institute)',
   'Other',
   'PhD student',
   'Postdoc',
   'Principal Investigator',
   'Research associate',
   'Undergraduate / Masters student']},
 'Engineering Science': {'counts': array([  0,   0,  47,  30,  52, 134,   0]),
  'values': ['Director (of the institute)',
   'Other',
   'PhD student',
   'Postdoc',
   'Principal Investigator',
   'Research associate',
   'Undergraduate / Masters student']},
 'Physics': {'counts': array([ 0,  0, 33, 38, 39, 57,  0]),
  'values': ['Director (of the institute)',
   'Other',
   'PhD student',
   'Postdoc',
   'Principal Investigator',
   'Research associate',
   'Undergraduate / Masters student']},
 'Life Science': {'counts': array([ 0,  0, 28, 29, 27, 33,  0]),
  'values': ['Director (of the institute)',
   'Other',
   'PhD student',
   'Postdoc',
   'Principal Investigator',
   'Research associate',
   'Undergraduate / Masters student']},
 'Ear

In [26]:
data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3],
       '2018'   : [2, 1, 3, 6, 7, 2],
       '2019'   : [1, 1, 2, 2, 2, 1],
       '2020'   : [5, 5, 4, 4, 5, 5]}

NameError: name 'fruits' is not defined

In [134]:
key = "careerLevel"

df = filter_dataframe(surveydata, include=[key, "researchArea"], exclude=[(key, to_exclude)])
all_areas = df[key].value_counts()
all_areas = all_areas.sort_index()
research_areas = list(df['researchArea'].value_counts().keys())
#data = {'All': all_areas.values, key: list(all_areas.keys()), 'researchArea': research_areas}

data = {'All': all_areas.values, key:list(all_areas.keys())}
x_keys = list(all_areas.keys())
y_keysl = ['All'] + research_areas
for area in research_areas:
    area_counts = df[df["researchArea"] == area][key].value_counts()
    area_counts = area_counts.sort_index()
    data[area] = area_counts.values

In [135]:
data

{'All': array([  0,   0, 130, 128, 148, 272,   0]),
 'careerLevel': ['Director (of the institute)',
  'Other',
  'PhD student',
  'Postdoc',
  'Principal Investigator',
  'Research associate',
  'Undergraduate / Masters student'],
 'Engineering Science': array([  0,   0,  47,  30,  52, 134,   0]),
 'Physics': array([ 0,  0, 33, 38, 39, 57,  0]),
 'Life Science': array([ 0,  0, 28, 29, 27, 33,  0]),
 'Earth Science': array([ 0,  0,  8, 11, 18, 32,  0]),
 'Chemistry': array([ 0,  0,  9, 12,  6,  4,  0]),
 'Other': array([0, 0, 1, 2, 3, 6, 0]),
 'Psychology': array([0, 0, 3, 2, 3, 2, 0]),
 'Mathematics': array([0, 0, 1, 4, 0, 4, 0])}

In [33]:

years = y_keysl

source = ColumnDataSource(data=data)
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
xkey = 'careerLevel'
p = figure(x_range=source.data[xkey], y_range=(0, 280), title="Question 1",
           height=350, toolbar_location=None, tools="hover, save, reset")


for i, y in enumerate(y_keys):
    p.vbar(x=dodge(xkey, -0.25, range=p.x_range), top=y, source=source,
       width=0.2, color=colors[i], legend_label=y)

#p.vbar(x=dodge('careerLevel', -0.25, range=p.x_range), top=years[0], source=source,
#       width=0.2, color="#c9d9d3", legend_label=years[0])

#p.vbar(x=dodge('careerLevel',  0.0,  range=p.x_range), top=years[1], source=source,
#       width=0.2, color="#718dbf", legend_label=years[1])

#p.vbar(x=dodge('careerLevel',  0.25, range=p.x_range), top=years[2], source=source,
#       width=0.2, color="#e84d60", legend_label=years[2])

p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1

p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.toolbar.logo = None
p.legend.location = "right"
p.legend.orientation = "vertical"
p.legend.click_policy="hide"
show(p)

NameError: name 'figure' is not defined

In [160]:
def bokeh_barchart(df, x, y, factors=None, figure=None, data_visible=[True], title='', 
                    width=0.1,  xlabel='', ylabel='Number of answers', palette=Category20c, 
                    fill_color=['blue'], legend_labels=None, description='For more information about the HMC survey click here.', redirect='https://helmholtz-metadaten.de/en/pages/structure-governance', 
                    tooltips=[('Data', '@{careerLevel}, @{top}'), ('Count', '@top')]):
    y_keys = y
    source = df
    help_t = HelpTool(description=description, redirect=redirect)
    tools = 'tap, wheel_zoom,box_zoom,undo,reset,save'
    fig = bokeh_figure(x_range=source.data[x], title=title, #y_range=(0, 280), 
           height=550, width=1200, toolbar_location='above', tools=tools)#, tooltips=[("$name @x", "@$name")])

    fig.add_tools(help_t)


    nvisible = len(y_keys)
    step = width + 0.05
    if nvisible%2 == 0:
        start = -step*nvisible/2 + step/2.0
    elif nvisible==1:
        start = 0.0
    else:
        start = nvisible//2 * -step
    
    position = [start + i*step for i in range(len(y))]        
    
    tooltips=[(f'{x}', f'@{x}')]
    bars = []
    for i, y in enumerate(y_keys):
        bar = fig.vbar(x=dodge(xkey, position[i], range=fig.x_range), top=y, source=source,
           width=width, color=fill_color[i], legend_label=y, fill_alpha=0.5, 
                       selection_fill_color='black', selection_fill_alpha=1.0,
                        nonselection_fill_alpha=0.2,
                       nonselection_fill_color="blue",
                       selection_line_color="black",
                       nonselection_line_alpha=1.0,
                       hover_fill_color='black', hover_fill_alpha=1.0,
                       hover_line_color="black"
                      )
                #hatch_pattern="x", hatch_alpha=0.5,hatch_weight= 1.0, hatch_scale=1, hatch_color= 'black')
        tooltips.append((f'{y}', '@{' + str(y) + '}'))
        bars.append(bar)

    print(tooltips)
    hover = HoverTool(tooltips=tooltips,renderers=bars)
    fig.add_tools(hover)
    fig.x_range.range_padding = 0.1
    fig.xgrid.grid_line_color = None
    fig.legend.location = "top_left"
    fig.legend.orientation = "horizontal"
    fig.y_range.start = 0
    fig.xaxis.major_label_orientation = 1
    fig.yaxis.axis_label = ylabel
    fig.xaxis.axis_label = xlabel
    fig.toolbar.logo = None
    fig.legend.location = "top_right"
    fig.legend.orientation = "vertical"
    fig.legend.click_policy="hide"
    #fig.outline_line_width = 7
    #fig.outline_line_alpha = 0.3
    #fig.outline_line_color = "navy"
    #legend= fig.legend.items
    #legend = Legend(items=leg_it)
    #fig.add_layout(legend, 'right')

    return fig

In [161]:
CD = ColumnDataSource(data=data)
colors = Category20c[len(y_keysl)]
fig = bokeh_barchart(CD, x='careerLevel', y=y_keysl, fill_color=colors, title='this is a title', tooltips=[('Data', '@{careerLevel}, @{All}'), ('Count', '@Physics')])
show(fig)

[('careerLevel', '@careerLevel'), ('All', '@{All}'), ('Engineering Science', '@{Engineering Science}'), ('Physics', '@{Physics}'), ('Life Science', '@{Life Science}'), ('Earth Science', '@{Earth Science}'), ('Chemistry', '@{Chemistry}'), ('Other', '@{Other}'), ('Psychology', '@{Psychology}'), ('Mathematics', '@{Mathematics}')]


In [62]:
CD.column_names

['All',
 'careerLevel',
 'Engineering Science',
 'Physics',
 'Life Science',
 'Earth Science',
 'Chemistry',
 'Other',
 'Psychology',
 'Mathematics']

In [ ]:
def bokeh_barchart_arrays(x, y, factors=None, figure=None, data_visible=[True], title='', 
                    width=0.9,  xlabel='Answers', ylabel='Number of answers', palette=Category20c, 
                    fill_color='blue', legend_labels=None):
    """Plot an interactive bar chart with bokeh"""

    if figure is None:
        if factors is not None:
            fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location='above',
               #x_range=FactorRange(factors=factors),
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
        else:
            fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location='above',
               tools='hover',
               #x_range=
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
    else:
        fig = figure

    
    nvisible = len(y)
    step = width + 0.05
    if nvisible%2 == 0:
        start = -step*nvisible/2 + step/2.0
    elif nvisible==1:
        start = 0.0
    else:
        start = nvisible//2 * -step
    
    position = [start + i*step for i in range(len(y))]
    for i, ydata in enumerate(y):
        print(x)
        print(ydata)
        ydata1 = df.data[ydata]
        x1 = df.data[x]
        #fig.vbar(x=dodge(x, position[i], range=fig.x_range), top=ydata, source=df,
        #           width=width, color=y_color[i], legend_label=leg_label[i])#, visible=data_visible[i])
        fig.vbar(x=x1, top=ydata1, width=width, line_color="white", fill_color=y_color[i])#factor_cmap('x', palette=palette, factors=factors, start=1, end=2))
    
    #fig.y_range.start = 0
    #fig.x_range.range_padding = 0.1
    #fig.xaxis.major_label_orientation = 1
    #fig.xgrid.grid_line_color = None
    fig.yaxis.axis_label = ylabel
    fig.xaxis.axis_label = xlabel
    #fig.xgrid.grid_line_color = None

    fig.toolbar.logo = None
    #fig.legend.location = "left"
    #fig.legend.orientation = "horizontal"
    #fig.legend.click_policy="hide"

    return fig

In [339]:
def bokeh_barchart2(df, x='x', y=['counts'], factors=None, figure=None, data_visible=[True], title='', 
                    width=0.9,  xlabel='Answers', ylabel='Number of answers', palette=Category20c, 
                    fill_color='color', legend_labels='legend_labels'):
    """Plot an interactive bar chart with bokeh"""

    if figure is None:
        if factors is not None:
            fig = bokeh_figure(plot_height=600, plot_width=1600,
               title=title,
               toolbar_location='above',
               #x_range=FactorRange(factors=factors),
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
        else:
            fig = bokeh_figure(plot_height=600, plot_width=1600,
               title=title,
               toolbar_location='above',
               tools='')#,#'hover',
               #x_range=
               #tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
    else:
        fig = figure
    '''
    if isinstance(x, list):
        if not 'color' in df.column_names:
            if isinstance(palette, dict):
                x_color = palette[len(x)]
        else:
            x_color = df.data['color']
        position = []
        step = width + 0.05
        nvisible = data_visible.count(True)
        if nvisible%2 == 0:
            start = -step*nvisible/2 + step/2.0
        elif nvisible==1:
            start = 0.0
        else:
            start = nvisible//2 * -step
        displayed_pos = [start + i*step for i in range(nvisible)]
        ind = 0
        for visible in data_visible:
            if not visible:
                position.append(0.0)
            else:
                position.append(displayed_pos[ind])
                ind = ind+1
        for i, data in enumerate(x):
            fig.vbar(x=dodge(data, position[i], range=fig.x_range), top=y[i], source=df,
                   width=width, color=x_color[i], legend_label=data, visible=data_visible[i])
    else:
        if isinstance(df, ColumnDataSource):
            xdata = df.data[x]
            if not 'color' in df.column_names:
                df.data['color'] = palette[len(xdata)]
        else:
            xdata = df[x]
            if not 'color' in df.columns:
                df['color'] = palette[len(xdata)]
    '''
    if isinstance(x, list):
        xdata = x
    else:
        xdata = df.data[x]
        
    if isinstance(legend_labels, list):
        leg_label = legend_labels
    else:
        leg_label = df.data[legend_labels]    

    if isinstance(fill_color, list):
        y_color = fill_color
    else:
        y_color = df.data[fill_color]
    
    if isinstance(y, list):
        y = y
    else:
        y = [y]
    nvisible = len(y)
    step = width + 0.05
    if nvisible%2 == 0:
        start = -step*nvisible/2 + step/2.0
    elif nvisible==1:
        start = 0.0
    else:
        start = nvisible//2 * -step
    
    position = [start + i*step for i in range(len(y))]
    print(position)
    for i, ydata in enumerate(y):
        print(x)
        print(ydata)
        ydata1 = df.data[ydata]
        x1 = df.data[x]
        #fig.vbar(x=dodge(x, position[i], range=fig.x_range), top=ydata, source=df,
        #           width=width, color=y_color[i], legend_label=leg_label[i])#, visible=data_visible[i])
        fig.vbar(x=x1, top=ydata1, width=width, line_color="white", fill_color=y_color[i])#factor_cmap('x', palette=palette, factors=factors, start=1, end=2))
    
    #fig.y_range.start = 0
    #fig.x_range.range_padding = 0.1
    #fig.xaxis.major_label_orientation = 1
    #fig.xgrid.grid_line_color = None
    fig.yaxis.axis_label = ylabel
    fig.xaxis.axis_label = xlabel
    #fig.xgrid.grid_line_color = None

    fig.toolbar.logo = None
    #fig.legend.location = "left"
    #fig.legend.orientation = "horizontal"
    #fig.legend.click_policy="hide"

    return fig

In [ ]:
# bokeh bar plot
def bokeh_barchart(df, x='value', y='counts', factors=None, figure=None, title='', width=0.9, 
                   xlabel='Answers', ylabel='Number of answers', palette=Category20c, fill_color='color'):
    """Plot an interactive bar chart with bokeh"""
    
    if isinstance(df, ColumnDataSource):
        xdata = df.data[x]
        if not 'color' in df.column_names:
            if not len(xdata) > 20:
                df.data['color'] = Category20c[len(xdata)] # ! if len(xdata)>20 this fails
    else:
        xdata = df[x]
        if not 'color' in df.columns:
            df['color'] = Category20c[len(xdata)] # ! if len(xdata)>20 this fails

    if figure is None:
        if factors is not None:
            fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location=None,
               x_range=FactorRange(factors=factors),
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
        else:
            fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location=None,
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
    
    #if factors is not None:
    #    fig.x_range=FactorRange(factors=factors)
    
    fig.vbar(x=x, top=y, width=width, source=df, line_color="white", fill_color=fill_color)#factor_cmap('x', palette=palette, factors=factors, start=1, end=2))
    fig.y_range.start = 0
    fig.x_range.range_padding = 0.1
    fig.xaxis.major_label_orientation = 1
    fig.yaxis.axis_label = ylabel
    fig.xaxis.axis_label = xlabel
    fig.xgrid.grid_line_color = None
    return fig

In [340]:
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]

fig = bokeh_barchart2(CD, x='careerLevel', y=y_keys, legend_labels=y_keys, fill_color=colors)
show(fig)

[-3.8000000000000003, -2.85, -1.9000000000000001, -0.9500000000000002, 0.0, 0.9499999999999997, 1.9, 2.85, 3.8000000000000003]
careerLevel
All
careerLevel
Engineering Science
careerLevel
Physics
careerLevel
Life Science
careerLevel
Earth Science
careerLevel
Chemistry
careerLevel
Other
careerLevel
Psychology
careerLevel
Mathematics


In [403]:
df = filter_dataframe(surveydata, include=["careerLevel", "researchArea"], exclude=[("careerLevel", to_exclude)])


In [391]:
df[df["researchArea"] == 'Physics'] or df["researchArea"] == 'Physics' ]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [399]:
df[df["researchArea"] == 'Physics']

,careerLevel,researchArea
2,PhD student,Physics
14,Research associate,Physics
15,Postdoc,Physics
17,Principal Investigator,Physics
19,PhD student,Physics
...,...,...
962,PhD student,Physics
980,Research associate,Physics
983,Postdoc,Physics
986,Research associate,Physics


In [405]:
re_field = list(df["researchArea"].value_counts().keys())
re_field

['Engineering Science',
 'Physics',
 'Life Science',
 'Earth Science',
 'Chemistry',
 'Other',
 'Psychology',
 'Mathematics']

In [406]:
re_field = list(df["researchArea"].value_counts().keys())
areas = ['Physics', 'Mathematics']
exclude = []
for field in re_field:
    if field not in areas:
        exclude.append(field)
for area in exclude:
    df = df[df["researchArea"] != area]
df

,careerLevel,researchArea
2,PhD student,Physics
14,Research associate,Physics
15,Postdoc,Physics
17,Principal Investigator,Physics
19,PhD student,Physics
...,...,...
983,Postdoc,Physics
984,Research associate,Mathematics
986,Research associate,Physics
989,Postdoc,Physics


In [214]:
a = df["careerLevel"].value_counts()

In [215]:
a = a.sort_index()

In [216]:
list(a.keys())

['Director (of the institute)',
 'Other',
 'PhD student',
 'Postdoc',
 'Principal Investigator',
 'Research associate',
 'Undergraduate / Masters student']

In [217]:
list(a.keys())

['Director (of the institute)',
 'Other',
 'PhD student',
 'Postdoc',
 'Principal Investigator',
 'Research associate',
 'Undergraduate / Masters student']

In [218]:
to_exclude = ['Other', 'Undergraduate / Masters student', 'Director (of the institute)']
df = filter_dataframe(surveydata, include=["careerLevel", "docStructured", "researchArea"], exclude=[("careerLevel", to_exclude)])

In [219]:
df.value_counts()

careerLevel         docStructured  researchArea       
Research associate  No             Engineering Science    51
                    Yes            Engineering Science    26
                    I am not sure  Engineering Science    17
                    Yes            Physics                17
Postdoc             No             Physics                16
                                                          ..
                    I am not sure  Psychology              1
PhD student         Yes            Psychology              1
                                   Earth Science           1
                    I am not sure  Psychology              1
Research associate  Yes            Psychology              1
Length: 72, dtype: int64

In [220]:
# data has to be in this format:

#data = {'fruits' : fruits,
#        '2015'   : [2, 1, 4, 3, 2, 4],
#        '2016'   : [5, 3, 3, 2, 4, 6],
#        '2017'   : [3, 2, 4, 4, 5, 3]}

In [221]:
df.keys()

Index(['careerLevel', 'docStructured', 'researchArea'], dtype='object')

In [222]:
All_research_areas = df[["careerLevel", "docStructured", 'researchArea']].

SyntaxError: invalid syntax (2957042116.py, line 1)

In [223]:
All_research_areas

careerLevel         docStructured  researchArea       
Research associate  No             Engineering Science    51
                    Yes            Engineering Science    26
                    I am not sure  Engineering Science    17
                    Yes            Physics                17
Postdoc             No             Physics                16
                                                          ..
                    I am not sure  Psychology              1
PhD student         Yes            Psychology              1
                                   Earth Science           1
                    I am not sure  Psychology              1
Research associate  Yes            Psychology              1
Length: 72, dtype: int64

In [224]:
All_research_areas.keys()

MultiIndex([(    'Research associate',            'No', 'Engineering Science'),
            (    'Research associate',           'Yes', 'Engineering Science'),
            (    'Research associate', 'I am not sure', 'Engineering Science'),
            (    'Research associate',           'Yes',             'Physics'),
            (               'Postdoc',            'No',             'Physics'),
            ('Principal Investigator',            'No',             'Physics'),
            (    'Research associate',            'No',             'Physics'),
            (           'PhD student',            'No', 'Engineering Science'),
            (               'Postdoc',            'No', 'Engineering Science'),
            ('Principal Investigator',           'Yes', 'Engineering Science'),
            (    'Research associate',            'No',       'Earth Science'),
            ('Principal Investigator',            'No', 'Engineering Science'),
            (           'PhD student',  

In [229]:
totals = surveydata["centerAffiliation"].value_counts()
totals

German Aerospace Center (DLR)                                                         298
Forschungszentrum Jülich (FZJ)                                                        102
German Cancer Research Center (DKFZ)                                                   79
Helmholtz-Zentrum Dresden-Rossendorf (HZDR)                                            76
Helmholtz-Zentrum Berlin für Materialien und Energie (HZB)                             64
Helmholtz-Zentrum Hereon                                                               38
Deutsches Elektronen-Synchrotron (DESY)                                                38
Karlsruhe Institute of Technology (KIT)                                                33
Helmholtz Centre for Infection Research (HZI)                                          21
German Center for Neurodegenerative Diseases (DZNE)                                    14
Helmholtz Zentrum München - German Research Center for Environmental Health (HMGU)     14
Helmholtz 

In [230]:
dfStart = surveydata.loc[surveydata["submitdate"] == "1980-01-01 00:00:00"]
# select columns for analysis: careerLevel vs. structured metadata
cols = ["careerLevel", "docStructured"]

df0 = dfStart[cols].dropna(how = "all", subset = cols).astype("category")

# calculate total answers by career level
totals = dict(df0["careerLevel"].value_counts())


In [231]:
totals = df0["careerLevel"].value_counts()


In [232]:
list(totals.keys())

['Research associate',
 'Principal Investigator',
 'Postdoc',
 'PhD student',
 'Other',
 'Undergraduate / Masters student',
 'Director (of the institute)']

In [233]:
# run evaluation

In [108]:
x = [str(i) for i in range(7)]
xs = [x,x,x,x,x]
xlabels = [[f'1 pli bla blub i' for i in x], [f'2 pli bla blub i' for i in x], [f'3 pli bla blub i' for i in x], [f'4 pli bla blub i' for i in x], [f'5 pli bla blub i' for i in x]]
ys = [[random.randint(0, 1200) for i in x], [random.randint(0, 1200) for i in x], [random.randint(0, 1200) for i in x],[random.randint(0, 1200) for i in x], [random.randint(0, 1200) for i in x]]

In [109]:
# create dashboard

In [110]:
# launch app

In [111]:
counts = totals.values
values = list(totals.keys())
df_test = ColumnDataSource(data=dict(value=values, counts=counts, factors=values))#, title=question))

NameError: name 'totals' is not defined

In [112]:
# bokeh bar plot
def bokeh_barchart(df, x='value', y='counts', factors=None, figure=None, title='', width=0.9, 
                   xlabel='Answers', ylabel='Number of answers', palette=Category20c, fill_color='color'):
    """Plot an interactive bar chart with bokeh"""
    
    if isinstance(df, ColumnDataSource):
        xdata = df.data[x]
        if not 'color' in df.column_names:
            if not len(xdata) > 20:
                df.data['color'] = Category20c[len(xdata)] # ! if len(xdata)>20 this fails
    else:
        xdata = df[x]
        if not 'color' in df.columns:
            df['color'] = Category20c[len(xdata)] # ! if len(xdata)>20 this fails

    if figure is None:
        if factors is not None:
            fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location=None,
               x_range=FactorRange(factors=factors),
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
        else:
            fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location=None,
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
    
    #if factors is not None:
    #    fig.x_range=FactorRange(factors=factors)
    
    fig.vbar(x=x, top=y, width=width, source=df, line_color="white", fill_color=fill_color)#factor_cmap('x', palette=palette, factors=factors, start=1, end=2))
    fig.y_range.start = 0
    fig.x_range.range_padding = 0.1
    fig.xaxis.major_label_orientation = 1
    fig.yaxis.axis_label = ylabel
    fig.xaxis.axis_label = xlabel
    fig.xgrid.grid_line_color = None
    return fig

In [113]:
def bokeh_barchart2(df, x='value', y='counts', factors=None, figure=None, data_visible=[True], title='', 
                    width=0.9,  xlabel='Answers', ylabel='Number of answers', palette=Category20c, fill_color='color'):
    """Plot an interactive bar chart with bokeh"""

    if figure is None:
        if factors is not None:
            fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location='right',
               x_range=FactorRange(factors=factors),
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
        else:
            fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location='right',
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
    else:
        fig = figure
    
    if isinstance(x, list):
        if isinstance(palette, dict):
            x_color = palette[len(x)]
        position = []
        step = width + 0.05
        nvisible = data_visible.count(True)
        if nvisible%2 == 0:
            start = -step*nvisible/2 + step/2.0
        elif nvisible==1:
            start = 0.0
        else:
            start = nvisible//2 * -step
        displayed_pos = [start + i*step for i in range(nvisible)]
        ind = 0
        for visible in data_visible:
            if not visible:
                position.append(0.0)
            else:
                position.append(displayed_pos[ind])
                ind = ind+1
        for i, data in enumerate(x):
            p.vbar(x=dodge(data, position[i], range=p.x_range), top=counts[i], source=df,
                   width=width, color=x_color[i], legend_label=data, visible=data_visible[i])
    else:
        if isinstance(df, ColumnDataSource):
            xdata = df.data[x]
            if not 'color' in df.column_names:
                df.data['color'] = palette[len(xdata)]
        else:
            xdata = df[x]
            if not 'color' in df.columns:
                df['color'] = palette[len(xdata)]
        fig.vbar(x=x, top=y, width=width, source=df, line_color="white", fill_color=fill_color)#factor_cmap('x', palette=palette, factors=factors, start=1, end=2))
    
    fig.y_range.start = 0
    fig.x_range.range_padding = 0.1
    fig.xaxis.major_label_orientation = 1
    fig.xgrid.grid_line_color = None
    fig.yaxis.axis_label = ylabel
    fig.xaxis.axis_label = xlabel
    fig.xgrid.grid_line_color = None


    #fig.legend.location = "left"
    #fig.legend.orientation = "horizontal"
    #fig.legend.click_policy="hide"

    return fig


def add_legend_outside(fig):
    """
    Move legend to the outside of a figure
    """
    #fig.legend.visible = False
    #legent_it = fig.legend.items
    #legend = Legend(items=legent_it)
    legend = fig.legend
    fig.add_layout(legend, 'right')
    return fig
# test
#df_test = pd.DataFrame(data=dict(value=xs[0], counts=ys[0]))
#fig = bokeh_barchart(df_test, factors=xs[0])
#show(fig)

# test2
#counts2 = sum(zip(*ys), ())
#factors = [(str(ques), ans) for ques in range(len(xs)) for ans in xs[0]] # factors have to be unique, and strings
#df_test3 = ColumnDataSource(data=dict(value=factors, counts=counts2))
#palette = Category20c[len(ys[0])]
#fig = bokeh_barchart(df_test3, factors=factors, fill_color=factor_cmap('value', palette=palette, factors=xs[0], start=1, end=2))
#show(fig)


In [114]:
dfStart = surveydata.loc[surveydata["submitdate"] == "1980-01-01 00:00:00"]
# select columns for analysis: careerLevel vs. structured metadata
cols = ["careerLevel", "docStructured"]#, "researchArea"]

df0 = dfStart[cols].dropna(how = "all", subset = cols).astype("category")


In [115]:
all_total = df0.value_counts()
all_total.sort_index()

careerLevel                      docStructured
Director (of the institute)      I am not sure     1
                                 No                6
                                 Yes               3
Other                            I am not sure     2
                                 No               13
                                 Yes              21
PhD student                      I am not sure    20
                                 No               32
                                 Yes              24
Postdoc                          I am not sure    30
                                 No               48
                                 Yes              18
Principal Investigator           I am not sure    19
                                 No               39
                                 Yes              39
Research associate               I am not sure    27
                                 No               88
                                 Yes              59

In [118]:

# calculate total answers by career level
totals = df0["careerLevel"].value_counts()
counts = totals.values
values = list(totals.keys())
df_test = ColumnDataSource(data=dict(value=values, counts=counts, factors=values))

In [119]:
df_test.data['factors']

['Research associate',
 'Principal Investigator',
 'Postdoc',
 'PhD student',
 'Other',
 'Undergraduate / Masters student',
 'Director (of the institute)']

In [120]:
isinstance(df_test, ColumnDataSource)

True

In [121]:
# test
#df_test = pd.DataFrame(data=dict(value=xs[0], counts=ys[0]))
fig = bokeh_barchart2(df_test, factors=df_test.data['factors'], fill_color='docStructured')
show(fig)

AttributeError: unexpected attribute 'plot_height' to figure, similar attributes are outer_height, height or min_height

In [106]:

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']
data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack
print(counts)

(2, 5, 3, 1, 3, 2, 4, 3, 4, 3, 2, 4, 2, 4, 5, 4, 6, 3)


In [107]:
# test

counts2 = sum(zip(*ys), ())
factors = [(str(ques), ans) for ques in range(len(xs)) for ans in xs[0]] # factors have to be unique, and strings
df_test3 = ColumnDataSource(data=dict(value=factors, counts=counts2))
palette = Category20c[len(ys[0])]
fig = bokeh_barchart(df_test3, factors=factors, fill_color=factor_cmap('value', palette=palette, factors=xs[0], start=1, end=2, ))
show(fig)

NameError: name 'ys' is not defined

In [249]:
df_test2 = ColumnDataSource(data=dict(value=x, counts=counts))

fig2 = bokeh_barchart(df_test2, factors=x, fill_color=factor_cmap('value', palette=palette, factors=years, start=1, end=2))
show(fig2)

In [152]:
# bokeh piechart    
def bokeh_piechart(df, x='value', y='counts', figure=None, radius=0.6, title=''):
    """Draw an interactive piechart with bokeh"""
    
    from math import pi
    
    ydata = np.array(df[y])
    df['angle'] = ydata / sum(list(ydata)) * 2 * pi
    
    if not 'color' in df.columns:
        df['color'] = Category20c[len(ydata)] # ! if len(xdata)>20 this fails
    df['percent'] = ydata / sum(ydata)


    if figure is None:
        fig = bokeh_figure(height=600, width=600,
               title=title,
               toolbar_location=None,
               tools='hover, tap',
               tooltips=[('Data', f'@{x}'),
                         ('Percent', '@percent{0.00%}'), 
                         ('Count', f'@{y}')])
    #fig.add_layout(Legend(), 'right')
    fig.wedge(x=0,
            y=1,
            radius=radius,
            start_angle=cumsum('angle', include_zero=True),
            end_angle=cumsum('angle'),
            line_color='white',
            fill_color='color',
            legend_field='xlabel',
            source=df, selection_fill_color='black', selection_fill_alpha=1.0,
                        nonselection_fill_alpha=0.2,
                       nonselection_fill_color="blue",
                       selection_line_color="black", fill_alpha=0.8,
                       nonselection_line_alpha=0.5, hover_fill_alpha=1.0,
                       hover_line_color="black", hover_line_width=5.0)
    
    fig.axis.axis_label = None
    fig.axis.visible = False
    fig.grid.grid_line_color = None
    return fig

In [153]:
# test
df_test = pd.DataFrame(data=dict(value=xs[0], counts=ys[0], xlabel=xs[0]))
fig = bokeh_piechart(df_test)
show(fig)

In [252]:
fig.legend.items

[LegendItem(id='1871', ...)]

In [253]:
# bokeh scatter plot
'''
def bokeh_scatter(x,
                  y=None,
                  *,
                  xlabel='x',
                  ylabel='y',
                  title='',
                  figure=None,
                  data=None,
                  saveas='scatter',
                  copy_data=False,
                  **kwargs):
    """Draw an interactive scatter with bokeh"""


    if isinstance(x, (dict, pd.DataFrame, ColumnDataSource)) or x is None:
        warnings.warn(
            'Passing the source as first argument is deprecated. Please pass in source by the keyword data'
            'and xdata and ydata as the first arguments', DeprecationWarning)
        data = x
        x = kwargs.pop('xdata', 'x')
        y = kwargs.pop('ydata', 'y')

    plot_data = process_data_arguments(data=data,
                                       x=x,
                                       y=y,
                                       copy_data=copy_data,
                                       single_plot=True,
                                       same_length=True,
                                       use_column_source=True)
    entry, source = plot_data.items(first=True)

    plot_params.set_defaults(default_type='function', name=entry.y)
    kwargs = plot_params.set_parameters(continue_on_error=True, **kwargs)

    p = plot_params.prepare_figure(title, xlabel, ylabel, figure=figure)

    plot_kwargs = plot_params.plot_kwargs(plot_type='scatter')
    res = p.scatter(x=entry.x, y=entry.y, source=source, **plot_kwargs, **kwargs)
    plot_params.add_tooltips(p, res, entry)

    if plot_params['level'] is not None:
        res.level = plot_params['level']

    plot_params.draw_straight_lines(p)
    plot_params.set_limits(p)
    plot_params.save_plot(p, saveas)

    return p
```

SyntaxError: EOF while scanning triple-quoted string literal (3054096253.py, line 51)

In [254]:
fig = bokeh_multi_scatter([[x] for x in xs[0]], [[y] for y in ys[0]], marker_size=[int(x)+6 for x in xs[0]])

In [255]:
questions = open(join(pwd, 'dashboard/data/questions.txt')).read().split()
QUESTION_MAP = {question : i for i, question in enumerate(questions)}


question_select = Select(title="Question", value="Question 1",
               options=questions)
filter_select = Select(title="Filter", value="All",
               options=open(join(pwd, 'dashboard/data/filters.txt')).read().split())
chart_select = Select(title="Filter", value="bar",
               options=['bar', 'pie'])
# Dublicate button


TOOLTIPS=[
    ("Title", "@title"),
    ("Answer", "@X"),
    ("Number of Answers", "@Y")
]


p = bokeh_figure(height=600, width=700, title="", toolbar_location='right', tooltips=TOOLTIPS, sizing_mode="scale_both")
source = ColumnDataSource(data=dict(x=[], y=[], color=[], title=[], year=[], revenue=[], alpha=[]))

FileNotFoundError: [Errno 2] No such file or directory: '/home/j.broeder/work/git/survey_dashboard/dashboard/dashboard/data/questions.txt'

In [256]:

def select_data():
    """Select the data to display"""
    question = question_select.value
    index = QUESTION_MAP(question)
    selected = ColumnDataSource(data=dict(x=xs[index], y=ys[index], title=question))
    return selected


def select_chart():
    """Select how the data should be displayed"""
    charttype = chart_select.value.strip()
    return charttype

def update():
    """Update the charts"""
    df = select_data()
    charttype = select_chart()

In [257]:
# develop interactive multi bar chart

In [162]:
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import dodge
from pprint import pprint
from bokeh.palettes import Category20

In [163]:
def update_bar(event):
    '''replot the bar chart, but leave out hidden items'''
    print('Python:Click')

In [164]:


width = 0.2
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017', '2018', '2019', '2020']
years_visible = [False, True, True, True, True, True]
years_color = Category20[len(years)]#["#c9d9d3","#718dbf", "#e84d60", "#e84d60", "#e84d60", "#e84d60"]
position = []#for i in range(years_visible.count(True))]
step = width+0.05
nvisible = years_visible.count(True)
print(nvisible%2)
if nvisible%2 == 0:
    start = -step*nvisible/2 + step/2.0
elif nvisible==1:
    start = 0.0
else:
    start = nvisible//2 * -step
displayed_pos = [start+i*step for i in range(nvisible)]
#print(displayed_pos, start, step)
ind = 0
for visible in years_visible:
    if not visible:
        position.append(0.0)
    else:
        position.append(displayed_pos[ind])
        ind = ind+1

#print(position)
#position = [-0.25, 0.0, 0.25]
data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3],
       '2018'   : [2, 1, 3, 6, 7, 2],
       '2019'   : [1, 1, 2, 2, 2, 1],
       '2020'   : [5, 5, 4, 4, 5, 5]}

# todo proper dynamic xrange, maybe use factor x_range instead always...
source = ColumnDataSource(data=data)
#x_axis = [ (fruit, year) for fruit in fruits for year in years ]

p = figure(x_range=fruits,#FactorRange(*x_axis), #fruits,#
           y_range=(0, 10), title="Fruit Counts by Year",
           height=600, toolbar_location=None, tools="")
for i, year in enumerate(years):
    p.vbar(x=dodge('fruits', position[i], range=p.x_range), top=year, source=source,
       width=width, color=years_color[i], legend_label=year, visible=years_visible[i])

p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"
p.legend.click_policy="hide"

show(p)

1


In [165]:
from bokeh.models import HelpTool

In [166]:
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, PrintfTickFormatter)
from bokeh.transform import transform


def bokeh_corr_plot(source, x='value', y='counts',  figure=None, title='', markersize='markersize',  xlabel='Answers', ylabel='Number of answers', alpha=None):
    """Plot an interactive circle with bokeh"""

    if figure is None:
        fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location='right',
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
    else:
        fig = figure
    if alpha is None:
        alpha = 1.0
    
    # this is the colormap from the original NYTimes plot
    colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
    mapper = LinearColorMapper(palette=colors, low=source.data[y].min(), high=source.data[y].max())
    
    fig.circle(source=source, x=x, y=y, radius=0.9)#,# fill_color={'field': y, 'transform': color_mapper}, 
    
    
    #           fill_alpha=alpha)
    #fig.rect(x=x, y=y, width=1, height=1, source=source,
    #   line_color=None, fill_color=transform('percentage', mapper))    
    
    
    #line_color='#7c7e71',
    #line_width=0.5,
    #line_alpha=0.5,
    #legend_group='region')
    
    fig.y_range.start = 0
    fig.x_range.range_padding = 0.1
    fig.xaxis.major_label_orientation = 1
    #fig.xgrid.grid_line_color = None
    fig.yaxis.axis_label = ylabel
    fig.xaxis.axis_label = xlabel


    #fig.legend.location = "left"
    #fig.legend.orientation = "horizontal"
    #fig.legend.click_policy="hide"

    return fig

In [167]:
dfStart = surveydata.loc[surveydata["submitdate"] == "1980-01-01 00:00:00"]


# select columns for analysis: careerLevel vs. structured metadata
cols = ["careerLevel", "docStructured"]

df0 = dfStart[cols].dropna(how = "all", subset = cols).astype("category")

# calculate total answers by career level
totals = dict(df0["careerLevel"].value_counts())

# exclude "Other", "undergrads" and "directors" answer from analysis
df = df0.loc[~df0.careerLevel.isin(['Other', 'Undergraduate / Masters student', 'Director (of the institute)'])]
df = df.loc[~df.careerLevel.isna()]
df = df.loc[~df.docStructured.isna()]

# calculate cross tab for the data frame
df_crosstab = pd.crosstab(df["careerLevel"],
                              df['docStructured'],
                              margins = False).reset_index().melt(id_vars = ["careerLevel"])

# include total answers by career level in cross tab
df_crosstab["total"] = df_crosstab["careerLevel"].map(totals).astype("int")

# calculate relative amount of answers by career level
df_crosstab["percentage"] = (df_crosstab["value"] / df_crosstab["total"]) * 100

# order "servFormat" categories
df_crosstab["docStructured"] = df_crosstab["docStructured"].astype("category").cat.set_categories(new_categories = ["Yes", "No", "I am not sure"], ordered = True)



In [168]:
df_crosstab['percentage'].min()

14.516129032258066

In [169]:
df_crosstab.keys()

Index(['careerLevel', 'docStructured', 'value', 'total', 'percentage'], dtype='object')

In [170]:
df_crosstab['docStructured']

0     I am not sure
1     I am not sure
2     I am not sure
3     I am not sure
4                No
5                No
6                No
7                No
8               Yes
9               Yes
10              Yes
11              Yes
Name: docStructured, dtype: category
Categories (3, object): ['Yes' < 'No' < 'I am not sure']

In [171]:
markersize = [5 for i in df_crosstab['percentage']]
df_crosstab['markersize'] = markersize
#careerlevel = [i for i in range(len(df_crosstab['careerLevel'] ))] 
#df_crosstab['careerLevel'] = careerlevel

#print(df_crosstab['careerLevel'])

In [172]:
selected = ColumnDataSource(df_crosstab)

In [173]:
selected.column_names

['index',
 'careerLevel',
 'docStructured',
 'value',
 'total',
 'percentage',
 'markersize']

In [174]:
fig = bokeh_corr_plot(source=selected, x='careerLevel', y='percentage', markersize='markersize')
show(fig)

AttributeError: unexpected attribute 'plot_height' to figure, similar attributes are outer_height, height or min_height

In [175]:
def heat_map(source, x='values', y='counts'):
    
    
    
    
    fig.axis.axis_line_color = None
    fig.axis.major_tick_line_color = None
    fig.axis.major_label_text_font_size = "7px"
    fig.axis.major_label_standoff = 0
    fig.xaxis.major_label_orientation = 1.0
    return fig

In [176]:
import pandas as pd
from bokeh.io import output_file, show
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, PrintfTickFormatter)
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data
from bokeh.transform import transform

output_file("unemploymemt.html")

data.Year = data.Year.astype(str)
data = data.set_index('Year')
data.drop('Annual', axis=1, inplace=True)
data.columns.name = 'Month'

# reshape to 1D array or rates with a month and year for each row.
df = pd.DataFrame(data.stack(), columns=['rate']).reset_index()

source = ColumnDataSource(df)

# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=df.rate.min(), high=df.rate.max())

p = figure(width=800, height=300, title="US unemployment 1948—2016",
           x_range=list(data.index), y_range=list(reversed(data.columns)),
           toolbar_location=None, tools="hover", x_axis_location="above")

p.rect(x="Year", y="Month", width=1, height=1, source=source,
       line_color=None, fill_color=transform('rate', mapper))

color_bar = ColorBar(color_mapper=mapper,
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"))

p.add_layout(color_bar, 'right')

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0

show(p)

In [179]:
from data.helpers.hcs_clean_dictionaries import HCS_colnamesDict


HCS_COLNAMES_REVERT_DICT = {val:key for key, val in HCS_colnamesDict.items()}

In [182]:
a= list(HCS_COLNAMES_REVERT_DICT.keys())
for en in a:
    print(en)

centerAffiliation
centerAffiliation_other
researchFieldHGF
researchArea
researchArea_other
researchAreaING
researchAreaING_other
researchAreaGEO
researchAreaGEO_other
researchAreaMATH
researchAreaMATH_other
researchAreaPHYS
researchAreaPHYS_other
researchAreaLIFE
researchAreaLIFE_other
researchAreaBIO
researchAreaBIO_other
researchAreaMED
researchAreaMED_other
researchAreaAGRI
researchAreaAGRI_other
researchAreaPSYCH
researchAreaPSYCH_other
researchAreaCHEM
researchAreaCHEM_other
yearsInResearch
careerLevel
careerLevel_other
orcid
fairFamiliarity
dataOrigin_MeasVsSim
dataOrigin_SelvVsReuse
dataAmount_lsf
lsfIdent_1
lsfIdent_2
lsfIdent_3
lsfIdent_4
lsfIdent_5
lsfIdent_6
lsfIdent_7
lsfIdent_8
lsfIdent_9
lsfIdent_10
lsfIdent_11
lsfIdent_other
dataGenMethod_1
dataGenMethod_2
dataGenMethod_3
dataGenMethod_4
dataGenMethod_5
dataGenMethod_6
dataGenMethod_other
dataGenMethodSpec_1_1
dataGenMethodSpec_1_2
dataGenMethodSpec_1_3
dataGenMethodSpec_2_1
dataGenMethodSpec_2_2
dataGenMethodSpec_2_3
da

In [273]:
source.data['Year']

array(['1948', '1948', '1948', '1948', '1948', '1948', '1948', '1948',
       '1948', '1948', '1948', '1948', '1949', '1949', '1949', '1949',
       '1949', '1949', '1949', '1949', '1949', '1949', '1949', '1949',
       '1950', '1950', '1950', '1950', '1950', '1950', '1950', '1950',
       '1950', '1950', '1950', '1950', '1951', '1951', '1951', '1951',
       '1951', '1951', '1951', '1951', '1951', '1951', '1951', '1951',
       '1952', '1952', '1952', '1952', '1952', '1952', '1952', '1952',
       '1952', '1952', '1952', '1952', '1953', '1953', '1953', '1953',
       '1953', '1953', '1953', '1953', '1953', '1953', '1953', '1953',
       '1954', '1954', '1954', '1954', '1954', '1954', '1954', '1954',
       '1954', '1954', '1954', '1954', '1955', '1955', '1955', '1955',
       '1955', '1955', '1955', '1955', '1955', '1955', '1955', '1955',
       '1956', '1956', '1956', '1956', '1956', '1956', '1956', '1956',
       '1956', '1956', '1956', '1956', '1957', '1957', '1957', '1957',
      

In [274]:


## helmholtz plot

In [275]:
svg_file_path = ''

In [276]:
svg_path_germany = "M1198.62 678.465c-1.12-2.261-3.37-6.211-2.25-8.474 1.13-2.255 0-5.924-2.82-5.078-2.82.844-5.08-1.133-6.78-1.976-1.01-.507-2.51-1.711-3.57-2.603l.81-.273-.89-.787c.6-.536 1.46-1.362 2.01-2.097 1.06-1.414.99-1.998.11-4.643-.23-.69-.35-1.331-.46-1.896-.28-1.48-.54-2.88-2.62-3.398-1.75-.439-1.99-1.4-2.5-3.5l-.05-.181c-.38-1.535-1.22-2.387-2.02-3.213-.53-.54-1.02-1.054-1.44-1.778-.85-1.48-.67-1.654 1-3.326 1.58-1.574 3.95-4.98 3.6-7.413-.07-.506-.16-.939-.24-1.332-.23-1.132-.35-1.7.17-2.743.11-.232.27-.501.43-.799 1.1-2.004 2.95-5.358.47-7.205-2.77-2.084-3.76-3.724-2.93-4.879.89-1.267 1.12-1.902.85-2.428a.897.897 0 0 0-.81-.48h-.02c-.27-.094-.67-.31-.77-.593v-.019c.05-.046.13-.104.19-.152.24-.187.57-.443.85-.864.5-.756.31-1.891-.05-2.704-.5-1.166-1.41-1.963-2.43-2.134-.23-.041-.51-.083-.8-.126-1.04-.153-3.81-.563-3.94-1.254-.13-.643-.41-1.306-.69-2.008-.4-.964-.85-2.055-1-3.299-.14-1.144-.7-1.865-1.14-2.445-.56-.726-.79-1.028-.34-1.923.79-1.585 1.26-2.11 2.04-2.981.28-.312.6-.664.96-1.108 1.41-1.699 1.17-2.632.66-4.014-.07-.201-.16-.419-.23-.661-.49-1.451-.23-3.065.57-3.673.36-.265.56-.658.57-1.106.02-1.384-1.97-3.268-3.65-4.606-.59-.475-1.59-.855-2.75-1.297-1.86-.712-4.18-1.598-4.96-3.001-.45-.815-.72-1.542-.96-2.184-.5-1.338-.92-2.494-2.6-2.973-.98-.281-1.56-.716-2.3-1.267-.62-.46-1.32-.982-2.37-1.506-2.06-1.03-3.67-3.143-3.68-3.165l-.25-.325-.41.026c-.01 0-.14.008-.34.008-1.31 0-2.38-.277-2.72-.706-.09-.109-.18-.281-.1-.643.19-.854.45-1.465.67-2.003.43-1.022.81-1.907.11-3.297-1.26-2.536-.64-3.38.03-3.83 2.97-1.98 4.92-3.554 5.82-4.676.15-.188.33-.405.52-.639 1.74-2.122 3.31-4.158 3.31-5.766 0-1.714-.29-5.717-.58-7.462-.22-1.368-.09-2.01 1.25-3.628.87-1.043 1.88-2.598 1.74-4.071a2.631 2.631 0 0 0-1-1.799c-1.17-.931-1.41-2.342-1.41-3.362v-.757h-.09c-.36-2.007-1.72-6.424-2.3-7.287-.24-.367-.66-1.655-1.02-2.79-.38-1.159-.76-2.36-1.17-3.296-.18-.425-.4-.873-.63-1.337-.79-1.612-1.68-3.438-1.3-4.957.49-1.956-.59-4.889-1.3-6.83-.18-.504-.34-.935-.4-1.159l-.01-.063-.03-.058c-.01-.026-1.07-2.447-1.21-4.27-.08-1.181-.51-1.973-.98-2.745l-.22-.362h-.48l-.01.003a3.968 3.968 0 0 0-.68-.066c-.9 0-1.63.385-2.12 1.114l-.16.246c-.4.624-.51.79-1.47.983-.42.083-.86.127-1.31.127-.92 0-1.91-.178-2.96-.528-.45-.148-.91-.28-1.36-.407-1.2-.339-2.33-.659-3.03-1.279-.65-.561-1.32-.877-1.97-1.181-.6-.287-1.23-.582-1.87-1.097-.71-.56-.85-1.14-.87-1.444.04-.03.07-.057.1-.084.4-.357.42-.372 1.56-.372l.56.005.42.003c1.12 0 1.74-.149 2.66-1.074.44-.431.67-.853.85-1.194.28-.525.35-.648.94-.699.75-.07 1.41-.07 2.11-.07.71 0 1.52 0 2.53-.075.56-.038 1.33-.06 2.27-.06.51 0 1.02.006 1.48.014l.81.014-.04-.813-.28-5.032-.02-.272-.18-.203c-.14-.15-3.45-3.729-4.62-4.607-.42-.315-.98-.907-1.57-1.53-.9-.948-1.92-2.023-2.98-2.729-.39-.257-.73-.517-1.06-.768-1.16-.876-2.25-1.703-4.57-2.17-.58-.115-1.04-.185-1.42-.242-1.36-.205-1.45-.218-2.62-1.975-.66-.991-1.15-2.005-1.55-2.823-.68-1.406-1.13-2.336-2.05-2.337-.12 0-.25.022-.37.062-.95.315-1.51 1.969-1.82 3.473-.22-.037-.6-.166-1.19-.561-.84-.561-1.58-.823-2.33-.823-1.2 0-2.24.723-3.09 1.431-1.44 1.201-3.92 1.522-5.16 1.522-.69 0-1.01.353-1.14.565-.39.642-.07 1.463.37 2.344-.38-.25-.89-.648-1.52-1.273-2.03-2.033-2.31-3.25-2.31-4.264 0-.32-.12-1.365-1.62-1.365-.26 0-.54.027-.82.055-.23.024-.46.049-.67.049a.916.916 0 0 1-.24-.022c-.38-.177-1.11-1.765-1.42-2.439-.14-.318-.27-.611-.4-.853 0-.012-.01-.021-.02-.034.22.116.43.247.64.379.71.445 1.44.905 2.29.905.35 0 .68-.078.99-.232.97-.487 2.14-1.827 2.34-3.041.1-.617-.06-1.173-.45-1.564a3.293 3.293 0 0 1-.49-.574l-.46-.754c.02-.007.04-.014.05-.02l.31-.127.11-.307.05-.129.03-.096c.13-.31.21-.572.29-.798.23-.691.27-.814 1.32-1.343.53-.265.91-.479 1.2-.645.48-.275.64-.369.97-.395-.04.136-.09.29-.14.461-.04.193-.14.592.12.926l.23.289.39.011c.46 0 .68-.289 1.27-1.061.42-.548.99-1.296 1.44-1.637a.712.712 0 0 1 .1-.068c.04.047.08.107.12.152.22.294.52.698 1.06.879.14.046.29.071.45.071.74 0 1.58-.557 2.18-1.048.27.458.46.895.48 1.113-1.12.873-.61 2.901-.49 3.308l.09.3.27.153c.15.084 1.5.815 2.51.815.73 0 1.26-.377 1.42-1.009.3-1.242.3-2.76 0-3.74.01-.002.01-.004.01-.005.27-.188.71-.503 1.06-1.188.74-1.488-.85-2.671-2.25-3.714-.26-.199-.53-.399-.78-.598-.66-.532-1.37-.978-2.14-1.358l-.28-.135-.29.097c-.31.103-.49.124-.59.124-.13 0-.18 0-.48-.503l-.16-.268-.3-.078c-.28-.072-.56-.299-.86-.696l-.1-.134-.15-.08c-.09-.049-.14-.08-.21-.122-.36-.203-.67-.29-1-.29-.61 0-1.08.305-1.53.603-.26.172-.53.351-.84.464a.856.856 0 0 1-.3.065c-.21 0-.41-.146-.77-.428a6.256 6.256 0 0 0-.82-.574c-.5-.28-.48-.882-.34-2.288l.01-.081c.15-1.487-.44-1.918-1.37-2.106-.12-.026-.26-.053-.42-.103-.63-.217-.99-.335-1.33-1.723-.19-.764-.43-1.708-1.34-1.708-.18 0-.34.044-.47.082-.12.035-.22.063-.34.063-.3 0-.69-.601-.8-1.715-.07-.703-.18-1.765-1.23-1.765-.24 0-.48.063-.78.14-.13.036-.28.074-.46.116-.31.066-.64.155-.91.338.3-.228.57-.496.81-.743.29-.288.62-.616.83-.698.23-.092.68-.278 2.33-2.385l.17-.21-.01-.269c-.01-.853.29-1.525 1.01-2.243 1.4-1.405 1.07-3.155 1.05-3.228l-.11-.534-.54-.067a8.265 8.265 0 0 0-.99-.051c-.92 0-2.25.119-3.39.687-.68.339-1.36.585-2.03.825-.95.344-1.86.671-2.6 1.225-1.29.969-1.9 2.997-1.97 3.224l-.41 1.43 1.4-.51 2.37-.865c.07.502.04 1.306-.48 2.351-.41.814-.68 1.514-.89 2.077l-.33.865-.21.234c-.05.064-.08.073-.15.073-.2 0-.41-.104-.63-.213-.26-.127-.55-.274-.9-.274-1.01 0-1.32 1.126-1.46 1.608-.28.979-.31 1.987-.34 2.874v.111l-.02.442c-.02.299-.06 1.092-.3 1.324-.8.809-1.63 1.841-1.63 3.362 0 1.185.71 2.075 1.76 2.215l.68.093.05-.212.1.032c.02.013.12.108.2.178.4.367 1.07.987 2.49.987.73 0 1.7 1.215 1.7 2.13 0 .222 0 .592-.86.592-2.77 0-4.24.223-4.58.282h-.63l-.12.691c-.05.632-.08 1.162-.08 1.618 0 1.737 1.17 2.394 2.11 2.923l.25.137c.16.092.32.182.48.272.79.426 1.46.799 1.96 1.485.56.794 1.13 1.192 1.85 1.697.29.203.61.429.98.721.43.328.84.49 1.26.49.97 0 1.57-.847 2.15-1.667.15-.213.3-.435.47-.653.12-.155.21-.253.27-.312a2.141 2.141 0 0 0 1.2.176c.48-.058 1.17-.271 2.16-.663l-.46.534-.13.158-.11.138c-1.36 1.469-3.3 2.302-3.85 2.438l-1.29.318c-.65-.266-1.41-.324-2.15-.336v-.16c-.99-.102-1.81-.24-2.2-.371-1.91-.633-3.18-1.409-3.5-2.132-.38-.84-1.34-2.913-2.22-3.98-.12-.211-.07-1.257.56-3.122l.13-.405-.27-.327c-.23-.679-1.01-.93-1.91-1.219-1.46-.474-1.88-.743-1.76-1.154.23-.81.01-1.342-.22-1.646-.4-.529-1.11-.799-2.11-.805l-.37-.002-.12.157-.73-.322a5.372 5.372 0 0 0-2.03-.381c-.29 0-.59.022-.9.065-1.97.282-6.13.558-7.8.558-.51 0-1.27.049-2.14.103-1.1.067-2.34.146-3.37.146-1.33 0-1.69-.132-1.77-.17-.28-.142-.62-.676-.9-1.103-.48-.749-1.02-1.599-1.94-1.599-.22 0-.44.054-.66.162-1.29.647-2.27 2.135-2.92 4.421-.44 1.51-.86 2.115-1.62 3.214-.16.242-.35.506-.55.802-1.24 1.871-2.38 3.599-2.38 5.225 0 .779-.5 1.17-1.45 1.85-.17.119-.34.24-.51.367-.83.626-2.7 2.137-4.2 3.352l-1.46 1.18c-1.58 1.263-2.76 3.68-3.06 4.588l-.04.103c-.16.495-.19.584-1.88.793-.58.073-1.21.086-1.88.101-1.84.039-3.93.083-5.58 1.485-.44.383-.83.751-1.19 1.087-.97.921-1.51 1.399-2.11 1.399-.14 0-.3-.028-.48-.087-.48-.158-.81-.433-1.21-.75-.62-.513-1.34-1.093-2.61-1.094-.34 0-.7.043-1.1.131-.89.196-1.46.208-1.91.215-.93.017-1.54.088-2.24 1.266-.29.475-.71.856-1.12 1.224-.51.462-.99.9-1.19 1.509l-.08.271.11.26c.23.527.45 1.306.31 2.169-.19-.597-.75-.943-1.28-1.207-.34-.171-.61-.247-.88-.247-.55 0-.89.333-1.18.629-.12.117-.27.261-.45.42-.97.804-.91 1.712-.86 2.441.01.17.02.35.02.54v.312l.22.222.42.423.41.414.51-.293c.07-.045.37-.222.76-.478-.18.208-.36.44-.52.708l-.13.222c-.15.237-.29.481-.36.559a.153.153 0 0 0-.03-.003l-.15.001c-.11 0-.23-.003-.37-.009-.23-.005-.51-.014-.85-.014-2.01 0-3.74 0-5.38 1.967-.85 1.025-1.6 2.476-1.13 3.479.24.502.73.79 1.36.79.35 0 .73-.083 1.19-.252 1.16-.436 1.45-.586 1.62-.664.05-.026.06-.033.27-.087v.01c-.04.416-.1.982-.1 1.729 0 .231.01.464.01.691.01.758.02 2.027-.2 2.247-.09-.009-.29-.03-.68-.231a25.15 25.15 0 0 1-2.48-1.42c-.71-.447-1.32-.833-2.1-1.147l-1.41-.562-.55-.218-.35.475c-.41.569-1.232 1.356-1.69 1.356-.132 0-.268-.135-.402-.405-.845-1.686.041-2.692.144-2.802l.964-1.015-1.371-.258c-1.288-.234-2.665-.795-2.787-1.133-.345-1.386-2.845-1.752-4.12-1.752-.89 0-2.099.144-2.792.834-1.097 1.098-4.947 3.838-6.236 4.352-.273.11-.63.255-.794.289-.111-.121-.293-.421-.506-.775-.788-1.316-2.168-1.662-3.383-1.968-.455-.113-.883-.222-1.248-.368-.57-.226-1.393-1.147-1.731-2.235-.152-.488-.257-1.183.101-1.723 1.057-1.584 1.99-2.201 3.328-2.201 2.051 0 4.612-1.348 4.968-3.152.204-.994.943-1.412 2.132-2.008l.293-.145c2.533-1.254 4.916-2.646 5.205-4.363.262-1.571-.304-3.636-.68-5.004-.07-.254-.132-.48-.178-.667-.261-1.045-.261-4.799-.261-6.032 0-.286.026-.609.051-.944.014-.186.028-.372.038-.56.278.196.588.299.912.299.494 0 .96-.238 1.313-.667.472-.577.659-1.408.495-2.224-.237-1.167-.237-1.167 1.082-2.046.429-.284.619-.307.666-.307.067 0 .176.096.349.255.266.243.632.577 1.204.721.364.09.695.13.984.163.689.083 1.145.137 1.772.765.352.353.725.532 1.113.532.532 0 .985-.352 1.185-.919.29-.829.04-2.16-1.005-2.995-.69-.551-.974-1.3-1.274-2.093-.223-.587-.454-1.193-.857-1.726-.197-.263-.328-.609-.467-.977-.353-.931-.834-2.202-2.771-2.688-2.208-.556-2.993-.818-4.314-1.609-.558-.334-1.034-.556-1.55-.556-.984 0-1.627.756-2.797 2.129-1.087 1.264-1.187 1.796-1.217 2.463-.012.279-.023.523-.298 1.072-.629 1.257-.496 1.878-.256 2.556.095.268.176.499.176.89v.642l.633.106c.025.005.26.041.615.041.664 0 1.951-.135 3.082-1.044.138-.111.245-.189.326-.245a6.188 6.188 0 0 0 .575.953c.412.553-.445 1.78-.904 2.439-.094.132-.178.253-.25.363a1.805 1.805 0 0 0-.305.786c-.417-.004-.791-.033-1.155-.061a15.331 15.331 0 0 0-1.243-.063c-.521 0-1.04.044-1.588.134-2.318.386-4.146 1.104-5.432 2.134-.483.387-.929.976-1.4 1.603-.979 1.296-2.089 2.764-3.719 2.764a2.65 2.65 0 0 1-.265-.012c-2.294-.213-3.56-1.664-4.399-2.624-.185-.211-.349-.399-.498-.547-1.038-1.04-5.477-2.748-7.477-3.038-1.266-.181-2.312-.922-3.32-1.637a19.871 19.871 0 0 0-1.198-.804c-.285-.171-2.825-1.657-4.676-1.657-.944 0-1.437.395-1.684.728-.286.379-.996.686-1.683.985-.5.216-1.008.437-1.44.721a3.567 3.567 0 0 0-.155-.266c-.725-1.086-.616-1.492-.602-1.533.006-.022.03-.089.21-.148.492-.167.795-.56.809-1.055.028-1.005-1.254-2.146-2.459-2.628-.246-.096-.485-.206-.716-.311-.72-.327-1.464-.662-2.291-.662-.497 0-.965.116-1.427.358-.837.019-3.672.167-4.85 1.344-.826.833-2.861 1.988-4.308 1.988-.298 0-.548-.048-.746-.148-.092-.046-.299-.163-.288-.267.046-.33.756-.986 1.509-1.137l.335-.064c1.498-.286 3.759-.719 4.971-2.842 1.016-1.775 1.511-3.93 1.511-6.588 0-1.356.135-2.138.322-3.221.074-.432.155-.906.239-1.468.078-.543.13-1.322-.304-1.865-.405-.505-.993-.555-1.422-.589a2.094 2.094 0 0 1-.475-.071c-.343-.111-.8-.223-1.238-.318a5.008 5.008 0 0 0 .896-2.24c.302-2.104.018-3.688-.942-5.298-.927-1.542-2.699-2.731-2.774-2.781l-1.176-.784v1.413c0 .021-.065 2.067-.935 2.067-1.502 0-2.711-.481-3.7-1.472-1.214-1.215-2.436-1.564-4.286-2.094l-.27-.077c-1.495-.425-3.898-1.733-4.818-2.25v-2.854l-1.137.664c-.14.082-3.456 2.017-4.89 3.17-.675.537-1.152 1.209-1.536 1.748a3.92 3.92 0 0 0-.126.178c-.239-.67-.426-1.666-.499-2.277l-.083-.665h-.669c-.083 0-2.069.023-3.687 1.967-.768.925-1.668 1.498-2.348 1.498-.243 0-.607-.061-.958-.585-.466-.7-.43-1.247-.394-1.826.037-.591.087-1.331-.547-1.962-.917-.923-2.801-1.916-4.765-1.916-1.506 0-4.308 0-5.543-.738-.222-.135-.424-.262-.613-.379-1.049-.661-1.805-1.138-3.44-1.138h-7.059c-.363 0-.822.084-1.354.178-.651.117-1.388.25-2.048.25-.687 0-1.131-.139-1.359-.427-.526-.653-1.226-.996-2.023-.996-.958 0-1.732.499-1.817.554l-.102.068-.075.097c-.021.026-.451.583-.865 1.357l-8.112-.236c-.515-.001-1.62-.179-1.884-.709-.202-.405-.202-.688-.202-1.25 0-.615 0-1.458-.296-2.812-.695-3.102-.23-3.347-.225-3.35a.704.704 0 0 1 .208-.026c.129 0 .259.014.393.029.189.016.38.034.573.034.895 0 1.57-.396 2.059-1.214.238-.396.712-.789 1.129-1.135.682-.567 1.324-1.103 1.237-1.876-.055-.473-.385-.867-.98-1.166-.41-.204-.792-.468-1.161-.726-.684-.473-1.331-.92-2.065-.92-.657 0-1.226.356-1.698 1.06-1.077 1.622-1.229 3.246-1.248 3.694l-.515 1.797c-1.373 2.474-2.283 4.554-2.861 7.78-.45 2.465-.384 4.596-.331 6.31.016.513.031.995.031 1.448 0 .485-.036 1-.072 1.499-.118 1.667-.239 3.39 1.213 3.749.29.076.526.108.741.108.591 0 .939-.268 1.127-.494.494-.594.547-1.599.185-3.584l-.065-.359c-.425-2.332-.425-2.332.159-3.674l.143-.332.084-.195c.203-.474.436-1.017.634-1.42.029.078.054.157.081.236.293.882.7 2.092 2.155 2.092.405 0 .868-.101 1.411-.304 1.738-.651 2.028-.818 2.775-1.298.191-.122.419-.268.736-.461.404-.241.886-.354 1.509-.354.415 0 .838.051 1.243.098.385.045.748.087 1.092.087h.031c-.08.537-.048 1.022.096 1.458.526 1.575 1.415 3.823 3.067 5.152.352.279.652.477.903.642.666.437.666.437.517 1.484-.343 2.443.468 4.413 1.973 4.792.571.141.835 1.331.835 2.091 0 .866.939 1.602 2.027 2.453.467.367 1.049.824 1.213 1.069.131.198.277.372.415.535.386.454.408.482-.097 1.117-.786.979-.6 1.757-.419 2.133.333.695 1.159 1.127 2.16 1.127.214 0 .432-.022.646-.061a5.538 5.538 0 0 1-.143.202c-.594.79-.447 1.604-.338 2.2.115.638.158.991-.212 1.455-.39.486-.723.671-1.075.868-.553.309-1.176.66-1.593 1.904-.5 1.497-.488 2.626.039 3.357.254.354.751.774 1.68.774.159 0 .331-.011.516-.038.155-.023.312-.031.465-.031.435 0 .883.069 1.316.137.429.068.833.132 1.227.132 1.107 0 1.825-.548 2.134-1.627.465-1.647.933-2.891 1.39-3.701.13.23.311.61.563 1.238.27.672.391 1.56-1.097 2.452-1.489.889-1.9 1.577-2.647 2.829l-.152.255c-.298.496-.448.706-.523.788a2.783 2.783 0 0 0-.129-.006c-.11-.01-.24-.016-.394-.016-.212 0-.549-.02-.963-.042-.7-.038-1.568-.086-2.445-.086-.852 0-1.534.046-2.082.138-.45.076-1.042.081-1.668.089-1.588.017-3.564.041-5.138 1.166l-1.972 1.417-1.09.781 1.231.528c.178.075 1.496.631 2.655.833-.152.117-.331.246-.543.397-.854.611-1.332.644-1.79.675-.511.036-1.145.079-1.76.895-1.2 1.609 1.288 3.955 1.798 4.407l.051.046.059.035.334.204c1.154.692 1.432.873 1.593 1.364.228.679.72.75.923.75.447 0 .74-.291 1.113-.663.403-.403.956-.954 1.811-1.321.485-.209 1.147-.315 1.967-.315 1.088 0 2.154.194 2.639.293.149.296.437 1.09.265 2.631-.173 1.553-.31 1.751-.635 2.231-.182.271-.41.604-.699 1.182-.902 1.809-1.03 2.713.071 4.182.351.466.367.607.394.861.03.281.073.665.382 1.282.451.901 1.22 1.418 2.113 1.418.687 0 1.407-.297 2.139-.882.297-.239.427-.251.427-.251.077.071.293.556.422.845.083.188.17.379.262.564.193.386.417.774.642 1.159.426.731.828 1.423.992 2.075.331 1.324.302 2.305.105 2.557-.012.016-.023.028-.099.028-.073 0-.206-.016-.419-.096-.416-.156-.78-.3-1.109-.432-1.007-.399-1.735-.688-2.47-.688-.432 0-.844.101-1.26.308-.224.115-.463.201-.686.283-.617.228-1.316.486-1.511 1.19-.182.656.219 1.259.539 1.656 1.115 1.401 1.909 2.461 2.716 4.338.176.412.327.785.467 1.13.553 1.359.95 2.342 1.952 3.346.943.939 2.014 1.809 3.453 1.809.411 0 .841-.073 1.275-.216.247-.082.539-.121.9-.121.603 0 1.285.109 2.006.229.779.127 1.585.259 2.365.259.268 0 .512-.016.747-.046 1.673.004 4.596.207 5.483.738.4.238.818.409 1.224.575 1.15.472 2.142.877 2.677 2.662.307 1.03.571 1.884.792 2.585-.728-.855-1.667-1.846-2.608-2.519-1.976-1.411-3.946-2.542-6.211-2.257-2.261.283-4.519 0-6.489 1.411-1.974 1.412-4.517 1.974-7.616 1.412-3.109-.566-5.083.847-6.778-1.132-1.694-1.976-4.231-5.644-6.208-3.387-1.976 2.257-3.385 3.953-3.671 5.648-.282 1.693-1.407 6.493-2.54 8.467-1.127 1.974-.282 5.93.847 7.623 1.133 1.694 1.693 2.258 1.693 4.518v4.515s-.848-2.257-1.973-3.389c-1.128-1.126 0 0-2.259-.564-2.256-.562-2.256 0-3.664-1.973-1.418-1.979-2.544-4.8-4.516-3.39-1.979 1.411-2.825 1.129-2.825 3.109 0 1.971 0 3.385-.564 4.797-.568 1.41 1.124 2.825 2.256 1.129 1.133-1.696 1.693-.284 1.981 2.258.278 2.538 0 3.668-1.419 5.363-1.408 1.695-2.818 2.826-4.793.843-1.978-1.974-1.415-3.944-3.954-3.944-2.538 0-3.952-3.394-2.254-3.956 1.694-.564 3.665 1.132 4.509-.564.852-1.696 1.131-2.824 0-4.798-1.124-1.98-1.69-4.522-2.535-5.647-.848-1.132-3.104 0-3.104-2.541s.559-3.103-1.413-3.673c0 0-2.254-.843-3.952-.843-1.691 0-3.385-.847-4.515.284-1.125 1.129-5.082.559-6.774 1.408-1.691.847-7.054 3.104-8.748 2.26-1.692-.851-4.795-1.692-6.774-1.131-1.976.564-6.207 0-7.899 1.695-1.694 1.692-4.805 3.951-5.93 5.646-1.125 1.695-.846 2.542.846 3.668 1.693 1.131 1.131 2.259-1.409 1.976-2.54-.28-2.54 0-3.108 2.26-.558 2.257-.558 4.516-1.408 6.775-.843 2.262-1.41 5.93 1.694 6.49 3.105.566 7.902-.28 9.315.29 1.406.563.278 0 0 2.252-.282 2.259 0 3.952-1.132 5.085-1.127 1.129.565 1.412.285 3.671-.285 2.255-1.133.842-1.693 2.821-.564 1.975-.279 2.823.281 5.644.565 2.822 1.412 7.058 0 9.602-1.412 2.539-3.384 7.336-5.083 9.312-1.691 1.984-2.538 5.651-2.259 8.189.286 2.543-.846 6.494-.846 7.623 0 1.124 0 4.521-1.972 3.383-1.974-1.123-3.39-.561-5.363-1.123-1.977-.568-3.386-1.699-4.794-1.136-1.415.568-5.085-.28-5.085 1.698 0 1.972-.28 3.672.568 4.234.845.565 1.971 1.131.28 1.978-1.693.845-1.975.564-1.975 2.255 0 1.694 0 4.236 1.408 4.803 1.414.564 5.082 1.971 6.776 2.539 1.693.566 4.516 1.409 4.795.282.286-1.129 1.975-3.669 2.261-1.41.284 2.256 1.976 5.079 2.823 5.64.845.566 1.124 3.395 0 4.803-1.127 1.412-2.539 5.366-1.979 6.209.313.47.883.935 1.332 1.261l-.002.001a5.281 5.281 0 0 1-2.564 1.094c-2.023.29-2.482 1.434-2.852 2.353-.078.194-.159.398-.259.599-.096.193-.128.224-.697.261-.506.033-1.202.077-1.9.599-.805.606-1.207 1.689-1.594 2.735-.228.614-.484 1.311-.745 1.571-.599.6-1.125.625-2.858.625-2.304 0-3.332 2.069-3.771 2.96a1.377 1.377 0 0 0-.331-.04c-.716 0-1.153.583-1.297.774-.436.58-.608 1.119-.517 1.643.148.886.97 1.497 2.11 2.265l.585.385c1.405.924 2.514 1.653 3.186 2.546.193.256.372.458.525.634.127.143.286.319.307.383-.001.01-.002.158-.313.679-.271.451-.484.785-.675 1.089-.416.654-.745 1.17-1.331 2.342-.532 1.071-1.921 2.156-2.559 2.305-.431-1.813-1.67-2.088-2.383-2.088-.421 0-.875.084-1.389.256-1.251.417-6.009 2.225-7.069 3.289-.509.508-1.594 1.224-2.416 1.224-.301 0-.515-.097-.688-.314-.874-1.087-2.102-1.815-3.061-1.815-.658 0-1.165.342-1.358.913-.274.83.321 1.474.8 1.992.271.293.642.695.592.893a.96.96 0 0 1-.038.128c-.151-.04-.518-.202-1.294-.819-1.262-1.015-2.943-2.187-4.731-2.187-.222 0-.443.018-.658.053a9.454 9.454 0 0 1-1.532.15c-.882 0-1.427-.196-1.622-.583-.145-.294-.278-.654-.408-1.004-.39-1.052-.831-2.244-1.957-2.244-.478 0-2.068.075-2.552 1.051-.139.278-.291.855.252 1.575.156.208.368.468.602.748.353.428 1.014 1.22 1.279 1.711-1.879-.013-2.245-.333-2.572-.992-.333-.662-.882-1.036-1.511-1.036-.803 0-1.609.611-2.106 1.598-.384.768-1.148 1.007-2.834 1.285-1 .17-1.686.701-1.884 1.464-.228.879.24 1.887 1.255 2.7.866.69 1.127 1.293 1.092 1.53-.017.125-.193.22-.334.278-1.585.634-2.256 1.587-2.328 1.693l-1.019 1.518 1.796-.354s.295-.053.731-.053c.418 0 1.047.05 1.665.297.89.358.934.931.934 2.684 0 1.802.324 2.788 2.17 3.526l.47.183c.421.163 1.158.446 1.444.664a1.29 1.29 0 0 1-.133.117c-.786.633-1.404 1.129-1.447 1.915-.04.745.456 1.389 1.384 2.318 1.729 1.727 2.388 2.696 2.618 3.845.156.78.461 1.286.758 1.775.327.54.666 1.098.942 2.214.127.507.281 1 .433 1.486.502 1.604.934 2.994.188 4.483-.743 1.495-.627 2.809-.533 3.861.062.71.116 1.323-.072 1.891-.217.652-.425.8-.923 1.155-.503.358-1.19.849-2.065 2.015-.924 1.228-1.596 2.366-2.192 3.371-.517.876-.965 1.631-1.464 2.261-.402.5-2.402 3.101-2.078 4.921.098.553.398.995.866 1.278a3.615 3.615 0 0 0 1.867.526c.523 0 1.027-.111 1.507-.283l-.194.139.625.632s.253.273.324.674c-.346.031-.84.09-1.48.347l-.41.159c-1.276.496-2.48.961-3.37 2.746-.394.785-.474.819-.74.931-.273.116-.687.29-1.314.794-1.124.893-1.757 2.227-2.223 3.199-.155.329-.295.619-.417.825-.316-.111-.932-.716-.932-1.179 0-1.523-.948-1.642-1.241-1.642-.452 0-.963.231-1.659.755-1.116.831-.714 1.948-.422 2.764.196.548.399 1.112.399 1.793 0 .414-.042.84-.082 1.252-.096.979-.187 1.904.362 2.509.292.321.699.483 1.213.483.253 0 .534-.038.857-.118 1.233-.308 2.007-.347 2.63-.38.497-.026.925-.049 1.382-.199a.545.545 0 0 1 .185-.031c.091 0 .138.022.148.029.007.038-.011.317-.357.835-.744 1.115-.585 1.942-.32 2.438.424.789 1.435 1.28 2.642 1.28.452 0 .553.069.556.072.098.134.098.821.098 1.428 0 .582.112 1.174.211 1.695.094.493.236 1.234.108 1.419-.021.028-.104.069-.249.091a2.946 2.946 0 0 1-.506.041c-.332 0-.667-.043-.992-.084a6.85 6.85 0 0 0-.874-.075c-.181 0-.737 0-1.159.388-.328.301-.488.733-.488 1.323 0 1.917-.107 2.228-.812 2.371-1.003.199-1.279.708-1.331 1.102-.121.887.865 1.581 1.714 2.005.27.136.524.231.755.312.155.057.37.135.425.18-.009 0 .039.201-.156.884-.409 1.432-.406 2.335.013 3.021.327.536.882.873 1.695 1.033.175.034.363.053.553.053.51 0 .983-.125 1.398-.236.306-.08.595-.156.808-.156.151 0 .258.018.411.269.516.865.561 1.001.777 1.654.118.353.277.832.566 1.608.275.735.27 1.191.264 1.598-.013 1.078.244 1.628 1.875 2.232a4.357 4.357 0 0 0 1.566.304c.454 0 .816-.081 1.081-.14.06-.011.129-.027.187-.036.027.05.06.132.101.253.186.56.432.91.617 1.141-.244.243-.675.62-.96.87-.86.751-1.376.948-1.753 1.092-.877.331-1.23.685-1.591 2.121-.514 2.055-.01 2.83.819 3.777.14.161.289.33.44.532.16.214.258.401.354.585.235.45.556 1.068 1.616 1.068.376 0 .878-.073 1.622-.239.278-.062.544-.092.786-.092 1.359 0 2.111.959 2.887 2.513.308.617.675 1.034.972 1.37.559.635.709.805.16 1.905-1.103 2.196-1.626 3.427-.537 5.245.118.198.273.387.45.574l-.008.035.34.292.001.001.964.837.02-.091c.189.141.372.29.411.377-.007.001-.033.076-.144.217l-.082.102c-.684.857-1.275 1.597-1.787 1.597-.153 0-.337-.061-.541-.185-.539-.322-1.069-.485-1.577-.485-1.198 0-2.17.852-2.893 2.534-.423.989-1.446 1.341-2.529 1.716-.648.225-1.26.435-1.744.8-1.384 1.031-2.397 3.092-1.354 5.177.042.084.117.233.185.377-.28.122-.662.317-1.107.673-.971.771-1.127 1.089-1.237 1.941-.048.369-.112.872-.419 1.792-.213.643-.491 1.282-.76 1.898-.79 1.826-1.607 3.712-.593 5.492.45.788.814 1.744 1.167 2.667.522 1.366 1.013 2.655 1.778 3.425.27.268.536.494.797.709.809.675 1.342 1.121 1.525 2.791.276 2.489 1.318 4.245 3.187 5.365.393.235.721.563 1.069.912.697.697 1.487 1.486 2.85 1.486.165 0 .34-.011.523-.037.546-.079.95-.133 1.225-.133.054 0 .095.002.125.005.024.097.051.315.051.787 0 1.83 1.737 3.015 4.426 3.015.879 0 1.619.262 1.887.669.17.261.157.63-.044 1.098-.405.944-.378 1.952-.355 2.842.033 1.311.016 2.015-.739 2.449-.372.213-.703.385-.998.537-1.321.688-2.195 1.142-2.479 3.116-.062.445-.099.838-.131 1.188-.106 1.172-.156 1.706-1.066 2.615-1.405 1.412-2.409 2.689-2.748 4.341-.08.411-.251.84-.431 1.291-.175.438-.356.891-.467 1.364l-.443 1.861 1.103-.729c.02.2.057.4.137.599.564 1.408.564 5.644 3.105 4.229 2.541-1.409 2.541.849 3.669 1.412 1.126.568 3.949 1.696 3.384 3.111-.565 1.407.284 2.54 1.412 3.106 1.127.563.847.845 0 1.691-.847.846-.28 1.694 1.127 3.389 1.412 1.694 3.957 3.389 3.957 5.082 0 1.694.84 3.669 2.252 3.104 1.415-.561 0 .565-.281 2.258-.28 1.697.565 2.263 2.546 2.541 1.97.282 1.689 1.413 2.815 1.413 1.131 0 2.54-1.981 2.26-3.109-.282-1.128-1.692-2.255-.563-2.538 1.128-.28 3.382-1.126 4.796-.28 1.408.843 3.389.28 4.234 2.253.847 1.983.847 2.827 1.41 4.519.568 1.694 2.541 1.694 2.541.284 0-1.412 1.125-1.129 2.54 0 1.41 1.124 4.798 2.542 6.206 1.695 1.412-.848 1.699.844 3.107-.571a40.822 40.822 0 0 0 1.525-1.616l.135 1.003 1.096-1.25c.222-.251.4-.457.51-.589.638-.218 1.964-.617 2.77-.617.242 0 .325.04.325.04.434.288 1.022.36 1.641.439 1.13.136 1.615.27 1.694.742.073.448.057.913.037 1.402-.046 1.162-.103 2.607 1.42 3.618.533.356 1.021.786 1.451 1.167.766.675 1.32 1.165 1.974 1.165.407 0 .764-.2 1.01-.564.078-.118.166-.174.271-.174.128 0 .289.084.353.273.174.521 1.542 3.326 3.403 3.326.427 0 .823-.145 1.181-.433.3-.237.716-.57.839-.612-.023.014.134.088.259.145.178.086.399.188.685.302.344.135.69.208 1.031.208.901 0 1.593-.473 2.205-.89.266-.181.517-.353.773-.483a.22.22 0 0 0 .034-.016c.077.074.192.234.28.356.325.445.77 1.054 1.601 1.389.154.06.302.121.441.178.646.271 1.256.524 1.94.524.696 0 1.344-.258 2.041-.814.473-.377.802-.553 1.04-.553.304 0 .729.292 1.404.833.842.676 2.025 1.242 3.068 1.742.813.391 1.579.758 2.029 1.095.168.127.334.259.504.391 1.032.813 2.205 1.739 3.905 1.739.797 0 1.971.772 2.899 1.569l-.304.363c-1.449 1.609-3.473 3.984-4.156 5.792-.25.672-.443 1.443-.65 2.26-.42 1.679-.945 3.769-1.951 4.056-2.111.603-3.083 1.23-4.329 2.792-.779.973-1.526 2.387-2.126 3.526-.302.572-.562 1.067-.739 1.333-.469.704-2.351 2.624-3.795 3.776-1.642 1.315-3.951 4.649-3.951 7.936 0 2.778-.698 4.6-2.266 5.907-1.346 1.118-1.294 2.429-1.248 3.583.027.643.051 1.25-.129 1.879-.254.886-.298 2.462-.346 4.131-.046 1.625-.123 4.35-.524 4.797l-.115.087c-1.118.836-2.083 1.559-2.69 3.988-.351 1.405-.929 2.324-1.73 3.59a42.171 42.171 0 0 0-1.3 2.161c-.732 1.315-1.175 2.416-1.53 3.307-.331.825-.593 1.485-.954 1.963-1.066 1.421-1.841 4.222-1.266 6.831.459 2.061 1.416 3.723 2.115 4.935.195.337.366.633.493.884.429.862.161 1.934-.705 2.799-1.179 1.176-1.848 2.912-1.683 4.289-.5.695-1.587 2.487-1.048 4.353.39 1.384.39 1.384-.358 2.466-.116.171-.249.361-.393.579-1.162 1.746-1.367 3.401-.639 5.221.303.75.223.886-.276 1.761-.162.278-.346.603-.548 1.003-.639 1.278-1.245 3.827 1.874 6.552 1.471 1.29 1.548 2.373 1.61 3.242.023.334.046.652.142.945.287.856 1.251 1.506 2.243 1.506.863 0 1.652-.47 2.218-1.323.44-.661.75-.916.904-1.015-.007.221-.481.832-.74 1.166-.33.432-.671.872-.911 1.352-.29.59-.244 1.26.127 1.86.504.81 1.478 1.3 2.605 1.3.622 0 1.269-.14 1.924-.43 1.018-.46 2.334-1.21 3.607-1.94 1.64-.95 3.885-2.234 4.549-2.234.208.092.447.954.561 1.364.264.96.563 2.04 1.511 2.28.545.14 1.675.15 2.135.15 1.167 0 7.101-.11 9.313-2.33 1.496-1.5 5.287-3.332 7.583-3.332.738 0 1.19.184 1.344.555 1.297 3.117 3.915 3.477 5.77 3.477 1.131 0 2.481-.15 4.376-.49l.778-.13-.174-.78c-.039-.19-.349-1.967 1.543-2.811.41-.181.841-.273 1.273-.273 1.261 0 2.252.793 2.462 1.101v.647h.757c.812 0 2.428-.548 2.72-2.618.07-.49.313-.888.548-1.274.313-.512.669-1.096.454-1.792-.194-.635-.793-1.136-2.001-1.687-.831-.38-1.511-.564-2.084-.564-1.154 0-1.711.73-2.117 1.265-.224.296-.438.576-.737.754-.587.352-1.382.565-2.12.565-.669 0-1.604-.168-2.268-.972-.512-.612-.678-1.516-.453-2.428.179-.708.579-1.296 1.001-1.464 1.529-.608 1.808-2.137 2.012-3.25.042-.229.081-.452.132-.652.141-.168 1.101-.365 1.674-.482 1.026-.209 2.189-.45 3.093-1.05.554-.371.91-.447 1.112-.447.523 0 .736.728.824 1.338.177 1.23.91 1.672 1.538 1.672.864 0 1.78-.834 2.102-2.159 2.018.79 3.539 1.78 3.549 2.189-.438.658-1.027.949-1.65 1.263-.811.399-1.727.858-2.125 2.048-.257.781-.026 1.341.215 1.677.748 1.032 2.472 1.115 2.983 1.115.407 0 .785-.04 1.093-.116a.451.451 0 0 1 .13-.017c.333 0 .791.281 1.196.531.505.307 1.027.629 1.598.629.558 0 1.019-.306 1.231-.822.332-.801-.019-1.961-.853-2.87.254.099.557.25.91.473 2.098 1.31 2.112 2.171 1.916 3.159-.127.631.012 1.232.39 1.687.417.513 1.08.793 1.863.793.796 0 1.684-.289 2.563-.84 1.864-1.167 3.65-1.757 5.312-1.757.494 0 .986.054 1.456.159 2.565.575 6.199 2.003 7.729 2.763.548.272 1.518.592 2.745.991 2.204.723 6.311 2.064 6.773 3.146 2.185 5.08 3.987 7.55 5.675 7.79.288.04.582.06.864.06 1.796 0 3.209-.85 4.738-1.87.187-.13.388-.19.61-.19.844 0 1.805.88 2.075 1.19l.423.48.548-.34 1.948-1.22c.579.19 1.745.52 2.653.52.741 0 1.262-.22 1.551-.65.336-.5.442-1.1.466-1.64l.088.01-.12-.96a1.25 1.25 0 0 0-.01-.06l-.004-.03c.234-1.17.79-2.9 1.378-3.346.194-.143.5-.233.823-.233.444 0 .85.159 1.035.404.048.064.196.265.067.715-.439 1.54-.087 2.45.288 2.95.426.56 1.11.88 1.924.88.194 0 .399-.02.615-.05.292-.05.63-.12.995-.19 1.06-.21 2.378-.46 3.284-.46.407 0 .582.05.64.08.155.25.304.62.462 1.02.336.84.755 1.9 1.58 2.72 1.12 1.12 2.837 1.71 3.951 1.71.219 0 .421-.03.603-.07.325.23 1.056 1.22 1.267 2.3.063.32.123.9-.178 1.2-.94.94-1.386 2.03-1.258 3.07.106.85.602 1.58 1.401 2.07.615.37 1.424.56 2.285.56 1.774 0 3.196-.82 3.626-2.1.097-.29.232-.29.289-.29.132 0 .354.09.543.32.173.22.433.7.247 1.54-.137.62-.243 1.2-.34 1.73-.269 1.47-.449 2.44-1.064 2.91-.791.59-1.144 1.53-.878 2.32.234.71.883 1.13 1.738 1.13.285 0 .599-.05.929-.14.196-.06.417-.11.653-.18 2.251-.6 6.021-1.62 7.177-4.5 1.056-2.64 3.164-4.42 3.998-4.64 1.118-.27 1.595-1.44 2.056-2.57.203-.49.412-1.01.668-1.44.702-1.17.923-3.43.484-4.93-.22-.74-.583-1.25-1.081-1.49-.795-.41-1.198-.61-.732-2.22.132-.47.223-.88.28-1.25.374.46.829.82 1.381.93.493.1 1.135.16 1.762.16 1.411 0 3.866-.27 4.464-2.07.111-.34.333-.72 1.064-.72.524 0 1.189.21 1.781.56.709.43 1.769.83 2.791 1.23 1.113.43 2.265.87 2.788 1.27.665.5 1.418.76 2.236.76.933 0 1.948-.33 3.109-.99.267-.15.567-.23.863-.23.584 0 1.034.29 1.129.58.121.36-.353.77-.561.92-.399.3-1.335 1-1.046 1.87.284.85 1.44.92 2.599.92.264 0 .532 0 .789-.01h.652c1.116 0 2.108 2.05 2.369 3.1.025.09.047.2.07.32.224 1.08.639 3.1 3.924 3.1.24 0 .48-.01.75-.03 2.34-.18 5.28-1.22 7.65-2.06.51-.17 1-.35 1.46-.51-.09.28-.14.57-.1.87.06.4.28.96 1.04 1.34l.43.22c.56.3.97.51 1.44.51.85 0 1.36-.66 2.18-1.81l.27-.37c1.17-1.66 1.78-2.45 2.87-2.45.12 0 .24 0 .38.02 1.9.28 3.01.55 3.02.55l.94.23v-4.75l-1.14 1.37c0-.02.01-.03.01-.04.27-2.49.45-2.71 2.07-2.71.54 0 1.11.03 1.71.06.61.04 1.24.08 1.85.08 1.08 0 2.44-.1 3.4-.88 1.08-.86.78-1.67.6-2.15-.16-.43-.29-.77.38-1.65 1.35-1.805 2.62-1.95 4.54-2.174l.59-.071c.19-.024.4-.033.63-.033.6 0 1.31.072 2.06.147.85.085 1.73.174 2.59.174 1.63 0 2.77-.328 3.59-1.029 1.63-1.396 6.38-2.95 8.37-2.95.12 0 .23.002.32.022.85.135 1.82.274 2.8.415 1 .142 2 .282 2.85.425.82.138 1.46.223 2.1.223.64 0 1.28-.085 2.1-.223l.09-.016c.94-.156 1.62-.267 1.91-.917.3-.674-.06-1.392-.92-2.761l-.32-.503c-1.31-2.067-1.71-2.84-.63-3.923.66-.655 1.18-1.304 1.65-1.877.52-.635.98-1.199 1.38-1.459.04.644-.21 2.016-.57 2.746-.11.22-.44.875-.09 1.431.14.236.44.513 1.04.513.6 0 1.52-.269 3.11-.898 1.41-.567 3.04-.909 4.36-.909 1.25 0 1.88.31 2.05.592.19.317.38.697.58 1.094.72 1.44 1.62 3.229 3.21 3.759.27.088.51.184.74.27.58.225 1.12.436 1.72.436.9 0 1.7-.462 2.59-1.503.33-.392.63-.759.9-1.106 1.08-1.357 1.67-2.109 3.41-2.299l.62-.072c.83-.096 1.53-.183 2.11-.183.82 0 1.3.168 1.73.597.33.336.51.757.73 1.248.37.869.84 1.947 2.33 2.588.28.12.44.217.54.283-.13.148-.36.361-.53.524-.17.155-.35.32-.53.502-.5.497-.69 1.184-.53 1.925.3 1.385 1.83 2.879 3.62 3.552.92.345 1.72 1.177 2.57 2.06 1.01 1.05 2.05 2.13 3.43 2.48 2.29.57 2.29.57 3.71.85.2.04.38.06.55.06 1.06 0 1.53-.69 1.94-1.3.21-.3.42-.62.74-.93 1.08-1.09.46-2.175.08-2.823-.22-.38-.42-.741-.42-1.108 0-.471-.1-.917-.2-1.35-.22-1.016-.4-1.819.57-2.962.18-.201.35-.398.52-.588 1.44-1.626 2.81-3.165 1.25-5.661-.26-.423-.54-.97-.83-1.548-.99-1.987-2.23-4.457-4.42-4.457-.65 0-1.34.229-2.03.69-.53.358-1.96 1.311-2.43 1.311-.14-.09-.37-.704-.56-1.199l-.11-.315c-.4-1.055-.22-1.299.31-2.023.34-.463.76-1.047 1.1-1.962.21-.569.65-1.21 1.11-1.883 1.22-1.813 2.75-4.067 1.08-6.693-.7-1.093-1.22-2.162-1.68-3.105-.88-1.809-1.57-3.234-2.95-3.694-1.46-.489-2.64-1.976-2.88-3.618-.27-1.901-1.62-2.437-3.06-2.997-.5-.199-1.02-.401-1.58-.684-.93-.465-1.53-1.092-1.75-1.817-.2-.666-.07-1.434.37-2.21.45-.787 1.06-1.128 1.82-1.562.87-.493 1.86-1.054 2.68-2.374.67-1.074.89-1.997 1.04-2.669.21-.873.27-1.061.81-1.246.36-.114.75-.169 1.24-.169.25 0 .5.013.73.026.23.01.45.026.65.026.34 0 1.24 0 1.73-.744.58-.866 3.56-4.038 5.56-4.788 1.49-.559 3.57-1.011 5.4-1.404.87-.192 1.71-.371 2.43-.551 2.24-.564 10.32-5.157 10.99-8.192.3-1.35.3-2.408.3-3.34 0-.888 0-1.657.26-2.54.55-1.942 1.11-3.624 1.11-3.64l.08-.251-.08-.247c-.31-.831-1.09-3.215-1.09-4.259 0-.125 0-.251.01-.354a6.38 6.38 0 0 0 1.35-.713c1.08-.718 1.31-1.093 1.53-1.58.1-.211.18-.393.56-.771.01-.013.03-.026.04-.036.32.368.86.919 1.98 1.339 1.02.382 1.97.595 2.81.785.99.222 1.85.416 2.36.801.29.218.66.624 1 1.023.81.904 1.64 1.845 2.64 1.845.34 0 .66-.111.95-.328.8-.606 1.06-1.349 1.31-2.068.19-.543.38-1.108.83-1.782.35-.523.65-.74.97-.97.75-.537 1.3-1.066 1.68-2.973.17-.837.4-1.621.6-2.311.49-1.706.91-3.177.16-4.436a7.431 7.431 0 0 0-.64-.905c-.41-.517-.56-.709-.38-2.293.07-.664.16-1.211.25-1.695.23-1.363.4-2.351-.33-3.806-.23-.467-.42-.865-.59-1.224-.47-.977-.81-1.684-1.46-2.563-.86-1.144-2.19-3.303-2.43-4.026-.18-.531-.62-.834-1.2-.834-.31 0-.62.076-.98.162-.4.096-.86.205-1.33.205-.15 0-.31-.011-.46-.037-1.26-.208-1.97-.95-2.82-2.937-1-2.309-3.28-2.96-5.29-3.534-.2-.057-.39-.084-.57-.085-1.18 0-1.6 1.068-1.85 1.709-.05.12-.11.282-.17.4-.06-.037-.13-.09-.23-.174-1.89-1.622-3.8-2.279-4.76-2.598.04-.265.15-.688.23-.991.28-1.047.6-2.233.22-3.181-.61-1.537-4.43-6.039-6.2-6.386-1.25-.254-4.5-.293-5.5-.298-.49-.462-1.36-1.402-1.52-2.214-.36-1.825-2.69-3.656-4.02-4.451-.68-.411-.93-.988-1.25-1.716-.21-.486-.43-.993-.8-1.48-1.14-1.536-2.27-2.857-3.45-4.03l-.26-.267c-1.4-1.409-3.74-3.773-6.73-3.773-.45 0-.91.056-1.35.166l-.51.131c-1.31.331-2.34.592-3.08.592-.67 0-1.1-.176-1.45-1.104-.96-2.572-4.06-5.57-6.63-5.57-.19 0-.28-.062-.34-.129-.55-.554-.54-2.473-.53-3.742 0-.79.01-1.474-.08-1.991-.27-1.667-1.91-2.453-2.69-2.737-.51-1.519-1.96-5.78-2.49-6.586-.34-.495-.88-.758-1.36-.988-.82-.388-.84-.484-.8-.707.23-1.108.83-4.067-1.12-4.553-1.56-.386-3.52-1.212-4.09-1.977-.84-1.109-1.06-1.503-.3-2.464 1.02-1.277 2.87-2.339 2.89-2.349l2.48-1.415h-2.86c.01 0-.19-.015-.27-.13-.08-.131-.09-.49.14-1.108l.1-.274c.76-2.023 1.04-2.781 2.31-4.307.84-1.01 1.65-2.199 1.34-3.375-.2-.777-.84-1.355-1.94-1.768-1.83-.688-1.97-1.117-1.98-1.164-.01-.032.03-.171.27-.418.53-.53.71-1.214.49-1.871-.35-1.074-1.78-1.995-3.46-2.233-1.35-.198-1.72-.491-2.95-1.457-.34-.267-.73-.578-1.21-.939-.92-.689-1.78-1.106-2.53-1.477-1.14-.556-1.9-.924-2.31-1.889-.7-1.638-1.8-2.168-2.67-2.591-.35-.167-.68-.324-.96-.539-.24-.177-.4-.579-.43-1.095-.06-.856.22-1.705.51-2.002.77-.762 1.08-2.221.72-3.322-.3-.911-1.01-1.489-1.99-1.63-.95-.133-1.31-.389-1.8-.743-.43-.3-.9-.641-1.74-.92-.52-.175-.87-.549-.9-.976-.03-.403.23-.774.72-1.017.52-.259.89-.683 1.06-1.227.38-1.228-.35-2.765-.68-3.369l-.09-.182c.51-.318 1.25-.696 1.91-.696 1.13 0 .85 2.261 1.69 3.385.84 1.132 2.54.562 3.39 1.413.84.85.28 1.411 0 2.259-.28.845.84.287 1.13 1.408.28 1.134 1.4 5.368 2.82 1.134 1.41-4.237 1.69-6.493 2.54-7.061.84-.561 1.41-2.257 1.41-3.383 0-1.13 2.26-1.978 3.11-2.26.84-.281 1.41-1.973 1.97-3.11.57-1.124 7.62-2.535 8.75-2.253 1.13.281 1.13 0 3.39-1.697 2.25-1.691 4.51-1.975 7.61-.28 3.11 1.695 2.27 1.977 2.27 1.977s3.1-.569 3.94-3.109c.85-2.538-.28-3.1.29-4.797.56-1.692 1.69 0 3.1-.565 1.41-.565 2.26-1.127 3.95-.565 1.7.565 3.11-.282 3.11-2.823 0-2.54 1.13-5.08 2.26-4.52 1.12.571 2.82 2.264 3.1.851.29-1.415.28-2.262 1.41-3.672 1.13-1.414 1.41-3.102 1.41-3.102s-.28 1.688 1.13 2.824c1.41 1.125 2.25 1.978 3.1 1.125.85-.847 1.14-.564 2.27-2.261 1.12-1.688 1.12-2.258 1.12-3.666 0-1.417-.85-3.382.56-3.105 1.42.281 2.26.571 4.24-.567 1.97-1.124 2.82-.28 4.23-.28 1.41 0 1.7-.844 3.67-1.41 1.98-.568 2.26.566 3.95 0 1.7-.568 2.82-.568 2.26-1.978-.56-1.413 0-3.945 3.39-4.797 3.39-.85 3.67.558 6.2-1.128 2.55-1.692 4.81-3.11 6.5-3.956 1.69-.847-.56-.561 1.41-2.258 1.98-1.695 2.54-1.126 4.8-1.695 2.25-.563 4.51-2.26 1.41-3.383-3.11-1.132-3.67-1.414-3.67-2.542 0-1.129-1.7-.847-2.26-1.695-.56-.846-.57-.846 0-2.824.56-1.975.28-1.975 2.54-1.692 2.26.283 2.26.283 3.39 1.408 1.13 1.132 1.69 1.416 2.82-.282 1.13-1.697 2.26-1.697 3.39-.563 1.13 1.129 3.94 1.977 4.51 3.107.57 1.126.57 2.821-.28 3.951-.84 1.132 0 2.825 1.98 1.694 1.97-1.132 2.53 1.41 1.4 2.821-1.12 1.41-.56 2.544 1.42 2.826 1.98.28 4.52 1.127 5.08 1.973.56.848 3.38.284 3.66-2.255.29-2.544.29-4.234 1.97-6.214 1.71-1.975 1.42-5.365 2.27-7.34.85-1.973 3.94-11.288 3.1-12.983-.85-1.697 0-3.958.57-5.928.56-1.974-1.13-5.076-2.27-7.339"

In [277]:
HGF_centers_orgi = [{'cor': (121,142), 'cor2': (87,100), 'Name': 'AWI'},
               {'cor': (467,45), 'cor2': (328,32), 'Name': 'CISPA'},
               {'cor': (118,200), 'cor2': (82,148), 'Name': 'DESY'},
               {'cor': (461,147), 'cor2': (333,120), 'Name': 'DKFZ'},
               {'cor': (362,87), 'cor2': (262,65), 'Name': 'DLR'},
               {'cor': (386,92), 'cor2': (283,70), 'Name': 'DZNE'},
               {'cor': (360,30), 'cor2': (259,22), 'Name': 'FZJ'},
               {'cor': (58,122), 'cor2': (38,78), 'Name': 'GEOMAR'}, # adjusted this by hand
               {'cor': (240,340), 'cor2': (184,250),'Name': 'GFZ'},
               {'cor': (421,146), 'cor2': (305,116),'Name': 'GSI'},
               {'cor': (204,208), 'cor2': (154,216),'Name': 'HGF'}, #adjusted by hand
               {'cor': (220,370),'cor2': (167,270), 'Name': 'HZB'},
               {'cor': (326,400), 'cor2': (230,285),'Name': 'HZDR'},
               {'cor': (245,235), 'cor2': (176,164),'Name': 'HZI'},
               {'cor': (556,157), 'cor2': (410,103), 'Name': 'HelmholtzMunich'},
               {'cor': (143,252), 'cor2': (105,186),'Name': 'Hereon'},
               {'cor': (510,124), 'cor2': (360,90),'Name': 'KIT'},
               {'cor': (193,369), 'cor2': (147,268),'Name': 'MDC'},
               {'cor': (312,324), 'cor2': (220,226),'Name': 'UFZ'}]

# adjusted this by hand, in the css there was some auto adjustment for right labels...
HGF_centers = [{'cor': (121,142), 'cor2': (87,100), 'Name': 'AWI'},
               {'cor': (467,45), 'cor2': (328,32), 'Name': 'CISPA'},
               {'cor': (118,200), 'cor2': (82,148), 'Name': 'DESY'},
               {'cor': (461,147), 'cor2': (333,120), 'Name': 'DKFZ'},
               {'cor': (362,87), 'cor2': (262,65), 'Name': 'DLR'},
               {'cor': (386,92), 'cor2': (283,70), 'Name': 'DZNE'},
               {'cor': (360,30), 'cor2': (259,22), 'Name': 'FZJ'},
               {'cor': (58,200), 'cor2': (38,78), 'Name': 'GEOMAR'}, 
               {'cor': (240,340), 'cor2': (184,250),'Name': 'GFZ'},
               {'cor': (421,146), 'cor2': (305,116),'Name': 'GSI'},
               {'cor': (204,345), 'cor2': (154,216),'Name': 'HGF'},
               {'cor': (220,370),'cor2': (167,270), 'Name': 'HZB'},
               {'cor': (326,400), 'cor2': (230,285),'Name': 'HZDR'},
               {'cor': (245,235), 'cor2': (176,164),'Name': 'HZI'},
               {'cor': (556,295), 'cor2': (410,103), 'Name': 'HelmholtzMunich'},
               {'cor': (143,252), 'cor2': (105,186),'Name': 'Hereon'},
               {'cor': (510,126), 'cor2': (360,90),'Name': 'KIT'},
               {'cor': (193,369), 'cor2': (147,268),'Name': 'MDC'},
               {'cor': (312,324), 'cor2': (220,226),'Name': 'UFZ'}]
#top, left

In [278]:
hgf_x = [center['cor'][1]+5 for center in HGF_centers] # +5 due to marker size, bokh centers cor, java right 
hgf_y = [-center['cor'][0]+648 for center in HGF_centers]
hgf_name = [center['Name'] for center in HGF_centers]

data_hgf = ColumnDataSource(data=dict(x=hgf_x, y=hgf_y, name=hgf_name))

In [279]:
#bokeh_scatter(data=data_hgf, x='x', y='y', tooltips=[('Name', '@name')], figure_kwargs={'width': 478, 'height':648})

In [280]:
print(min(points_x), max(points_x), min(points_y), max(points_y))
print(min(hgf_x), max(hgf_x), min(hgf_y), max(hgf_y))

NameError: name 'points_x' is not defined

In [281]:
from svg.path import parse_path

In [282]:
path_p = parse_path(svg_path_germany)

In [283]:
points_x = []
points_y = []
for segment in path_p:
    start = segment.start
    # we only need start, otherwise all are doubled
    points_x.append(start.real)
    points_y.append(-start.imag)
points_x.append(points_x[0])
points_y.append(points_y[0])
points_y = [poi-min(points_y) for poi in points_y]
points_x = [poi-min(points_x) for poi in points_x]

In [284]:
fig1 = bokeh_figure(width=478, height=648, title="HGF centers", x_axis_type= None, y_axis_type=None, 
                    x_range=(min(points_x), max(points_x)), y_range=(min(points_y), max(points_y)),
           toolbar_location=None, outline_line_color = None, tools="hover")#, x_axis_location=None, y_axis_location=None)
tooltips=[('Name', '@name')]
fig1.scatter(source=data_hgf, x='x', y='y', fill_alpha=0.8, size=12, fill_color='white', line_width=2, line_color='black')
fig1.line(x=points_x, y=points_y, color='black')
# display labels, /Names
# activate tab tool, for callbacks
# hover tool should show name,click hint and?
#

#fig.toolbar_location = None
#fig1.outline_line_color = None
#fig.grid.grid_line_color = None
#fig.axis.axis_line_color = None
#fig.axis.major_tick_line_color = None
#fig.x_axis.visible
show(fig1)

In [285]:
# world map with repos, for FAIR score

In [286]:
bokeh_scatter(data=data_hgf, x='x', y='y', figure= fig1)#, tooltips=[('Name', '@name')], figure=fig1)#
#figure_kwargs={'width': 478, 'height':648})

AttributeError: 'str' object has no attribute 'append'

In [407]:


# Create and deploy interactive data applications

from IPython.display import IFrame
IFrame('https://demo.bokeh.org/sliders', width=900, height=500)



In [177]:
from bokeh.io import show
from bokeh.layouts import gridplot
from bokeh.models import Div


def mk_div(**kwargs):
    return Div(text='<div style="background-color: green; width: 100px; height: 100px;">DIV</div>', **kwargs)


divs = [mk_div(), mk_div(), mk_div(),
        mk_div(), mk_div(margin=(10, 20, 30, 40)), mk_div(),
        mk_div(), mk_div(), mk_div()]

show(gridplot(divs, ncols=3))

In [ ]:
HCS_colnamesDict = {
        "PERBG1":"centerAffiliation",
        "PERBG1_other":"centerAffiliation_other",
        "PERBG2":"researchFieldHGF",
        "PERBG3": "researchArea",
        "PERBG3_other":"researchArea_other",
        "PERBG3ING":"researchAreaING",
        "PERBG3ING_other":"researchAreaING_other",
        "PERBG3GEO":"researchAreaGEO",
        "PERBG3GEO_other":"researchAreaGEO_other",
        "PERBG3MATH":"researchAreaMATH",
        "PERBG3MATH_other":"researchAreaMATH_other",
        "PERBG3PHYS":"researchAreaPHYS",
        "PERBG3PHYS_other":"researchAreaPHYS_other",
        "PERBG3LIFE":"researchAreaLIFE",
        "PERBG3LIFE_other":"researchAreaLIFE_other",
        "PERBG3BIO":"researchAreaBIO",
        "PERBG3BIO_other":"researchAreaBIO_other",
        "PERBG3MED":"researchAreaMED",
        "PERBG3MED_other":"researchAreaMED_other",
        "PERBG3AGRI":"researchAreaAGRI",
        "PERBG3AGRI_other":"researchAreaAGRI_other",
        "PERBG3PSYCH":"researchAreaPSYCH",
        "PERBG3PSYCH_other":"researchAreaPSYCH_other",
        "PERBG3CHEM":"researchAreaCHEM",
        "PERBG3CHEM_other":"researchAreaCHEM_other",
        "PERBG4":"yearsInResearch",
        "PERBG6":"careerLevel",
        "PERBG6_other":"careerLevel_other",
        "PERBG7":"orcid",
        "PERBG8":"fairFamiliarity",
        "RSDP1_1A2":"dataOrigin_MeasVsSim",
        "RSDP1_3A4":"dataOrigin_SelvVsReuse",
        "RSDP1b_1":"dataAmount_lsf",
        "RSDP1c_1":"lsfIdent_1",
        "RSDP1c_2":"lsfIdent_2",
        "RSDP1c_3":"lsfIdent_3",
        "RSDP1c_4":"lsfIdent_4",
        "RSDP1c_5":"lsfIdent_5",
        "RSDP1c_6":"lsfIdent_6",
        "RSDP1c_7":"lsfIdent_7",
        "RSDP1c_8":"lsfIdent_8",
        "RSDP1c_9":"lsfIdent_9",
        "RSDP1c_10":"lsfIdent_10",
        "RSDP1c_11":"lsfIdent_11",
        "RSDP1c_other":"lsfIdent_other",
        "RSDP2_1":"dataGenMethod_1",
        "RSDP2_2":"dataGenMethod_2",
        "RSDP2_3":"dataGenMethod_3",
        "RSDP2_4":"dataGenMethod_4",
        "RSDP2_5":"dataGenMethod_5",
        "RSDP2_6":"dataGenMethod_6",
        "RSDP2_other":"dataGenMethod_other",
        "RSDP2b_1_1":"dataGenMethodSpec_1_1",
        "RSDP2b_1_2":"dataGenMethodSpec_1_2",
        "RSDP2b_1_3":"dataGenMethodSpec_1_3",
        "RSDP2b_2_1":"dataGenMethodSpec_2_1",
        "RSDP2b_2_2":"dataGenMethodSpec_2_2",
        "RSDP2b_2_3":"dataGenMethodSpec_2_3",
        "RSDP2b_3_1":"dataGenMethodSpec_3_1",
        "RSDP2b_3_2":"dataGenMethodSpec_3_2",
        "RSDP2b_3_3":"dataGenMethodSpec_3_3",
        "RSDP2b_4_1":"dataGenMethodSpec_4_1",
        "RSDP2b_4_2":"dataGenMethodSpec_4_2",
        "RSDP2b_4_3":"dataGenMethodSpec_4_3",
        "RSDP2b_5_1":"dataGenMethodSpec_5_1",
        "RSDP2b_5_2":"dataGenMethodSpec_5_2",
        "RSDP2b_5_3":"dataGenMethodSpec_5_3",
        "RSDP2b_6_1":"dataGenMethodSpec_6_1",
        "RSDP2b_6_2":"dataGenMethodSpec_6_2",
        "RSDP2b_6_3":"dataGenMethodSpec_6_3",
        "RSDP2b_7_1":"dataGenMethodSpec_7_1",
        "RSDP2b_7_2":"dataGenMethodSpec_7_2",
        "RSDP2b_7_3":"dataGenMethodSpec_7_3",
        "RSDP3_1":"dataFormats_1",
        "RSDP3_2":"dataFormats_2",
        "RSDP3_3":"dataFormats_3",
        "RSDP3_4":"dataFormats_4",
        "RSDP3_5":"dataFormats_5",
        "RSDP3_6":"dataFormats_6",
        "RSDP3_7":"dataFormats_7",
        "RSDP3_8":"dataFormats_8",
        "RSDP3_9":"dataFormats_9",
        "RSDP3_10":"dataFormats_10",
        "RSDP3_11":"dataFormats_11",
        "RSDP3_12":"dataFormats_12",
        "RSDP3_13":"dataFormats_13",
        "RSDP3_14":"dataFormats_14",
        "RSDP3_15":"dataFormats_15",
        "RSDP3_other":"dataFormats_other",
        "RSDP7":"dataInPublication",
        "RSDP4":"dataGatherTime",
        "RSDP8":"experimentDuration_sub",
        "RSDP11":"dataAnalDuration_sub",
        "RSDP10":"longtermStorage",
        "DTPUB6_1":"pubAmount",
        "DTPUB1b_1":"pubMethod_1",
        "DTPUB1b_2":"pubMethod_2",
        "DTPUB1b_3":"pubMethod_3",
        "DTPUB1b_other":"pubMethod_other",
        "DTPUB5_1":"pubRepo_1",
        "DTPUB5_2":"pubRepo_2",
        "DTPUB5_3":"pubRepo_3",
        "DTPUB5_4":"pubRepo_4",
        "DTPUB5_5":"pubRepo_5",
        "DTPUB3_1":"pubMotivation_1",
        "DTPUB3_2":"pubMotivation_2",
        "DTPUB3_3":"pubMotivation_3",
        "DTPUB3_4":"pubMotivation_4",
        "DTPUB3_5":"pubMotivation_5",
        "DTPUB3_6":"pubMotivation_6",
        "DTPUB3_7":"pubMotivation_7",
        "DTPUB3_other":"pubMotivation_other",
        "DTPUB4a_0":"pubObstaclesA_0",
        "DTPUB4a_1":"pubObstaclesA_1",
        "DTPUB4a_2":"pubObstaclesA_2",
        "DTPUB4a_3":"pubObstaclesA_3",
        "DTPUB4a_4":"pubObstaclesA_4",
        "DTPUB4a_5":"pubObstaclesA_5",
        "DTPUB4a_6":"pubObstaclesA_6",
        "DTPUB4a_7":"pubObstaclesA_7",
        "DTPUB4a_other":"pubObstaclesA_other",
        "DTPUB4b_0":"pubObstaclesB_0",
        "DTPUB4b_1":"pubObstaclesB_1",
        "DTPUB4b_2":"pubObstaclesB_2",
        "DTPUB4b_3":"pubObstaclesB_3",
        "DTPUB4b_4":"pubObstaclesB_4",
        "DTPUB4b_5":"pubObstaclesB_5",
        "DTPUB4b_6":"pubObstaclesB_6",
        "DTPUB4b_7":"pubObstaclesB_7",
        "DTPUB4b_other":"pubObstaclesB_other",
        "RDMPR1_0":"dataStorage_0",
        "RDMPR1_1":"dataStorage_1",
        "RDMPR1_2":"dataStorage_2",
        "RDMPR1_3":"dataStorage_3",
        "RDMPR1_other":"dataStorage_other",
        "RDMPR3_0":"docMethod_0",
        "RDMPR3_1":"docMethod_1",
        "RDMPR3_2":"docMethod_2",
        "RDMPR3_3":"docMethod_3",
        "RDMPR3_other":"docMethod_other",
        "RDMPR7_2":"docMetadata_2",
        "RDMPR7_3":"docMetadata_3",
        "RDMPR7_4":"docMetadata_4",
        "RDMPR7_5":"docMetadata_5",
        "RDMPR7_6":"docMetadata_6",
        "RDMPR7_7":"docMetadata_7",
        "RDMPR7_8":"docMetadata_8",
        "RDMPR7_9":"docMetadata_9",
        "RDMPR7_other":"docMetadata_other",
        "RDMPR8_2":"docDigital_2",
        "RDMPR8_3":"docDigital_3",
        "RDMPR8_4":"docDigital_4",
        "RDMPR8_5":"docDigital_5",
        "RDMPR8_6":"docDigital_6",
        "RDMPR8_7":"docDigital_7",
        "RDMPR8_8":"docDigital_8",
        "RDMPR8_9":"docDigital_9",
        "RDMPR8_10":"docDigital_other",
        "RDMPR9_2":"docAuto_2",
        "RDMPR9_3":"docAuto_3",
        "RDMPR9_4":"docAuto_4",
        "RDMPR9_5":"docAuto_5",
        "RDMPR9_6":"docAuto_6",
        "RDMPR9_7":"docAuto_7",
        "RDMPR9_8":"docAuto_8",
        "RDMPR9_9":"docAuto_9",
        "RDMPR9_10":"docAuto_other",
        "RDMPR4":"docStructured",
        "RDMPR5":"docDefSchema",
        "DTPUB7_0":"pubMetadata_0",
        "DTPUB7_1":"pubMetadata_1",
        "DTPUB7_21":"pubMetadata_21",
        "DTPUB7_22":"pubMetadata_22",
        "DTPUB7_23":"pubMetadata_23",
        "DTPUB7_24":"pubMetadata_24",
        "DTPUB7_31":"pubMetadata_31",
        "DTPUB7_32":"pubMetadata_32",
        "DTPUB7_33":"pubMetadata_33",
        "DTPUB7_41":"pubMetadata_41",
        "DTPUB7_42":"pubMetadata_42",
        "DTPUB7_43":"pubMetadata_43",
        "DTPUB7_44":"pubMetadata_44",
        "DTPUB7_51":"pubMetadata_51",
        "DTPUB7_52":"pubMetadata_52",
        "DTPUB7_61":"pubMetadata_61",
        "DTPUB7_62":"pubMetadata_62",
        "DTPUB7_71":"pubMetadata_71",
        "DTPUB7_72":"pubMetadata_72",
        "DTPUB7_81":"pubMetadata_81",
        "DTPUB7_82":"pubMetadata_82",
        "DTPUB7_83":"pubMetadata_83",
        "DTPUB7_91":"pubMetadata_91",
        "DTPUB7_92":"pubMetadata_92",
        "DTPUB7_93":"pubMetadata_93",
        "DTPUB7_other":"pubMetadata_other",
        "RDMPR6_1":"docStandards_1",
        "RDMPR6_2":"docStandards_2",
        "RDMPR6_3":"docStandards_3",
        "RDMPR6_4":"docStandards_4",
        "RDMPR6_5":"docStandards_5",
        "RDMPR6_6":"docStandards_6",
        "RDMPR6_7":"docStandards_7",
        "RDMPR6_8":"docStandards_8",
        "RDMPR6_9":"docStandards_9",
        "RDMPR6_10":"docStandards_10",
        "RDMPR6_11":"docStandards_11",
        "RDMPR6_12":"docStandards_12",
        "RDMPR6_13":"docStandards_13",
        "RDMPR6_14":"docStandards_14",
        "RDMPR6_15":"docStandards_15",
        "RDMPR6_16":"docStandards_16",
        "RDMPR6_17":"docStandards_17",
        "RDMPR6_18":"docStandards_18",
        "RDMPR6_19":"docStandards_19",
        "RDMPR6_20":"docStandards_20",
        "RDMPR6_21":"docStandards_21",
        "RDMPR6_22":"docStandards_22",
        "RDMPR6_23":"docStandards_23",
        "RDMPR6_24":"docStandards_24",
        "RDMPR6_25":"docStandards_25",
        "RDMPR6_26":"docStandards_26",
        "RDMPR6_other":"docStandards_other",
        "RDMPR10_1":"software_1",
        "RDMPR10_2":"software_2",
        "RDMPR10_3":"software_3",
        "RDMPR12_0":"docMotivation_0",
        "RDMPR12_1":"docMotivation_1",
        "RDMPR12_2":"docMotivation_2",
        "RDMPR12_3":"docMotivation_3",
        "RDMPR12_4":"docMotivation_4",
        "RDMPR12_5":"docMotivation_5",
        "RDMPR12_6":"docMotivation_6",
        "RDMPR12_other":"docMotivation_other",
        "RDMPR11_0":"docObstacles_0",
        "RDMPR11_1":"docObstacles_1",
        "RDMPR11_2":"docObstacles_2",
        "RDMPR11_3":"docObstacles_3",
        "RDMPR11_4":"docObstacles_4",
        "RDMPR11_5":"docObstacles_5",
        "RDMPR11_6":"docObstacles_6",
        "RDMPR11_7":"docObstacles_7",
        "RDMPR11_8":"docObstacles_8",
        "RDMPR11_9":"docObstacles_9",
        "RDMPR11_other":"docObstacles_other",
        "SERVC1_0":"servNeeds_sub_0",
        "SERVC1_1":"servNeeds_sub_1",
        "SERVC1_2":"servNeeds_sub_2",
        "SERVC1_3":"servNeeds_sub_3",
        "SERVC1_4":"servNeeds_sub_4",
        "SERVC1_5":"servNeeds_sub_5",
        "SERVC1_6":"servNeeds_sub_6",
        "SERVC1_7":"servNeeds_sub_7",
        "SERVC1_8":"servNeeds_sub_8",
        "SERVC1_9":"servNeeds_sub_9",
        "SERVC1_other":"servNeeds_sub_other",
        "SERVC2_1":"servFormat_1",
        "SERVC2_2":"servFormat_2",
        "SERVC2_3":"servFormat_3",
        "SERVC2_4":"servFormat_4",
        "SERVC2_5":"servFormat_5",
        "SERVC2_6":"servFormat_6",
        "SERVC3":"feedback"
        }

In [33]:
HCS_COLNAMES_REVERT_DICT = {val:key for key, val in HCS_colnamesDict.items()}


NameError: name 'HCS_colnamesDict' is not defined

In [161]:
def filter_dataframe(df: pd.DataFrame, include: list=None, exclude: List[Tuple[str, list]]=None, exclude_nan=True) -> pd.DataFrame:
    """
    Filter pandas dataframe

    example:
    ```
    to_exclude = ['Other', 'Undergraduate / Masters student', 'Director (of the institute)']
    df = filter_dataframe(surveydata, include=["careerLevel", "docStructured", "researchArea"], exclude=[("careerLevel", to_exclude)])
    ```
    """
    
    if include is not None:
        df = df[include].dropna(how = "all", subset = include).astype("category")
    
    for key, val in exclude:
        df = df.loc[~df[key].isin(val)]
    
    if exclude_nan:
        for key in df.keys():
            df = df.loc[~df[key].isna()]
    return df

def get_all_values(df: pd.DataFrame, key: str, sort_as=None) -> dict:
    """
    Count all values of a given key in a data frame and 
    return these values in a dictionary sorted.
    """
    all_areas = df[key].value_counts()
    all_areas = all_areas.sort_index()
    data = {'All': all_areas.values, key:list(all_areas.keys())}
    return data

def prepare_data_research_field(df: pd.DataFrame, key:str, key2:str='researchArea', sort_as=None):# -> dict, list:
    """Creates a dict dictionary with data in the form needed by the plotting functions
    
    We prepare several outputs, i.e y_keys because they can have different length and one should be able to create a 
    ColumnDataSource by ColumnDataSource(data=data)
    :param df: [description]
    :type df: pd.DataFrame
    :param key: [description]
    :type key: str

    example:
    prepare_data_research_field(df, key=careerLevel)
        {'Cum. Sum': array([  0,   0, 130, 128, 148, 272,   0]),
         'careerLevel': ['Director (of the institute)',
          'Other',
          'PhD student',
          'Postdoc',
          'Principal Investigator',
          'Research associate',
          'Undergraduate / Masters student'],
         'researchArea': ['Engineering Science',
          'Physics',
          'Life Science',
          'Earth Science',
          'Chemistry',
          'Other',
          'Psychology',
          'Mathematics'],
         'Engineering Science': array([  0,   0,  47,  30,  52, 134,   0]),
         'Physics': array([ 0,  0, 33, 38, 39, 57,  0]),
         'Life Science': array([ 0,  0, 28, 29, 27, 33,  0]),
         'Earth Science': array([ 0,  0,  8, 11, 18, 32,  0]),
         'Chemistry': array([ 0,  0,  9, 12,  6,  4,  0]),
         'Other': array([0, 0, 1, 2, 3, 6, 0]),
         'Psychology': array([0, 0, 3, 2, 3, 2, 0]),
         'Mathematics': array([0, 0, 1, 4, 0, 4, 0])}

    """
    # Create Dataframe where index represents the sort order wanted 
    
    
    all_areas = df[key].value_counts()
    # bring in proper order
    if sort_as is None:
        all_areas = all_areas.sort_index()
    else:
        all_areas = sort_dataframe_by(all_areas, key, sort_as)
        
    research_areas = list(df[key2].value_counts().keys())
    data = {'Cum. Sum': all_areas.values, key:list(all_areas.keys()), 'x_value': list(all_areas.keys())}
    y_keys = ['Cum. Sum'] + research_areas
    for area in research_areas:
        area_counts = df[df[key2] == area][key].value_counts()
        if sort_as is None:
            area_counts = area_counts.sort_index()
        else:
            all_areas = sort_dataframe_by(all_areas, key, sort_as)
        data[area] = area_counts.values
    
    return data, y_keys


def sort_dataframe_by(df, key, value_order):
    if isinstance(df, pd.Series):
        # Sorting of Series is different, we cannot add an additional column
        df = pd.DataFrame({'counts': df.values, key: list(df.index)})
    df_mapping = pd.DataFrame({key: value_order})
    sort_mapping = df_mapping.reset_index().set_index(key)
    df[f'{key}_num'] = df[key].map(sort_mapping['index'])
    sorted_df = df.sort_values(f'{key}_num')
        
    return sorted_df

In [162]:
a = pd.Series(data=[1,2,3], index = ['a','b','c'])

In [163]:
a.values

array([1, 2, 3])

In [164]:
ad = pd.DataFrame({'key': a.values, 'index':a.index})
ad

,key,index
0,1,a
1,2,b
2,3,c


In [165]:
data_filters = ['All', 'Physics']
q_index = 'careerLevel'
to_exclude = []
sort_as = [
          'Other',
           'Undergraduate / Masters student',
          'PhD student',
          'Postdoc',
          
          'Research associate',
       'Principal Investigator',
    'Director (of the institute)'
          ]
df = filter_dataframe(survey_data, include=[q_index, "researchArea"], exclude=[(q_index, to_exclude)])
data_all = get_all_values(df, q_index, sort_as=sort_as)
exclude = []
for field in re:
    if field not in data_filters:
        exclude.append(field)
for area in exclude:
    df = df[df["researchArea"] != area]
data, y_keys = prepare_data_research_field(df, q_index, sort_as=sort_as)
if 'All' in data_filters:
    data['All'] = data_all['All']

#data_filters = y_keys
ydata_spec = {}
colors = []
for key in data_filters:
    colors.append(ra_colors[key])

ydata_spec['y_keys'] = data_filters
ydata_spec['colors'] = colors
ydata_spec['legend_labels'] = data_filters
selected = ColumnDataSource(data=data)
ydata_spec = ColumnDataSource(data=ydata_spec)

In [166]:
data

{'Cum. Sum': array([[11, 'Other', 0],
        [4, 'Undergraduate / Masters student', 1],
        [33, 'PhD student', 2],
        [38, 'Postdoc', 3],
        [57, 'Research associate', 4],
        [39, 'Principal Investigator', 5],
        [3, 'Director (of the institute)', 6]], dtype=object),
 'careerLevel': ['counts', 'careerLevel', 'careerLevel_num'],
 'x_value': ['counts', 'careerLevel', 'careerLevel_num'],
 'Physics': array([57, 39, 38, 33, 11,  4,  3]),
 'Chemistry': array([0, 0, 0, 0, 0, 0, 0]),
 'Earth Science': array([0, 0, 0, 0, 0, 0, 0]),
 'Engineering Science': array([0, 0, 0, 0, 0, 0, 0]),
 'Life Science': array([0, 0, 0, 0, 0, 0, 0]),
 'Mathematics': array([0, 0, 0, 0, 0, 0, 0]),
 'Other': array([0, 0, 0, 0, 0, 0, 0]),
 'Psychology': array([0, 0, 0, 0, 0, 0, 0]),
 'All': array([ 22,  65, 130, 128, 148, 272,  24])}

In [142]:
all_areas = df[q_index].value_counts()
all_areas = all_areas.sort_index()
# Create Dataframe where index represents the sort order wanted
df_mapping = pd.DataFrame({q_index: sort_as})
sort_mapping = df_mapping.reset_index().set_index(q_index)
# pd.Series behave strange, therefore put in a dataframe
all_areas1 = pd.DataFrame({'counts': all_areas.values, q_index: list(all_areas.keys())})
all_areas1[f'{q_index}_num'] = all_areas1[q_index].map(sort_mapping['index'])
all_areas1.sort_values(f'{q_index}_num')



,counts,careerLevel,careerLevel_num
1,11,Other,0
6,4,Undergraduate / Masters student,1
2,33,PhD student,2
3,38,Postdoc,3
5,57,Research associate,4
4,39,Principal Investigator,5
0,3,Director (of the institute),6


,index
careerLevel,
Other,0
Undergraduate / Masters student,1
PhD student,2
Postdoc,3
Research associate,4
Principal Investigator,5
Director (of the institute),6


,counts,careerLevel
0,3,Director (of the institute)
1,11,Other
2,33,PhD student
3,38,Postdoc
4,39,Principal Investigator
5,57,Research associate
6,4,Undergraduate / Masters student


In [133]:
all_areas[f'{q_index}_num'] = all_areas[q_index].map(sort_mapping['index'])
all_areas

,counts,careerLevel,careerLevel_num
0,3,Director (of the institute),6
1,11,Other,0
2,33,PhD student,2
3,38,Postdoc,3
4,39,Principal Investigator,5
5,57,Research associate,4
6,4,Undergraduate / Masters student,1


In [134]:
all_areas.sort_values(f'{q_index}_num')


,counts,careerLevel,careerLevel_num
1,11,Other,0
6,4,Undergraduate / Masters student,1
2,33,PhD student,2
3,38,Postdoc,3
5,57,Research associate,4
4,39,Principal Investigator,5
0,3,Director (of the institute),6


In [126]:
sortedd = all_areas.sort_values(f'{q_index}_num', axis='index')
all_areas

,counts,careerLevel,careerLevel_num
0,3,Director (of the institute),6
1,11,Other,0
2,33,PhD student,2
3,38,Postdoc,3
4,39,Principal Investigator,5
5,57,Research associate,4
6,4,Undergraduate / Masters student,1


In [123]:
 all_areas.sort_values?

In [109]:
all_areas.keys()

Index(['counts', 'careerLevel', 'careerLevel_num'], dtype='object')

In [110]:
all_areas.values

array([[3, 'Director (of the institute)', 6],
       [11, 'Other', 0],
       [33, 'PhD student', 2],
       [38, 'Postdoc', 3],
       [39, 'Principal Investigator', 5],
       [57, 'Research associate', 4],
       [4, 'Undergraduate / Masters student', 1]], dtype=object)

In [104]:
all_areas1 = pd.DataFrame({'counts': all_areas.values, q_index: list(all_areas.keys())})
all_areas1 = all_areas1.sort_values(axis=f'{q_index}_num')

ValueError: Per-column arrays must each be 1-dimensional

In [56]:
all_areas.sort_values?
pd.DataFrame(all_areas)

,careerLevel
Director (of the institute),3
Other,11
PhD student,33
Postdoc,38
Principal Investigator,39
Research associate,57
Undergraduate / Masters student,4


In [57]:
all_areas.values

array([ 3, 11, 33, 38, 39, 57,  4])

In [58]:
all_areas.keys

<bound method Series.keys of Director (of the institute)         3
Other                              11
PhD student                        33
Postdoc                            38
Principal Investigator             39
Research associate                 57
Undergraduate / Masters student     4
Name: careerLevel, dtype: int64>

ImportError: cannot import name 'Wordcloud' from 'wordcloud' (/home/j.broeder/work/envs/survey_db/lib/python3.8/site-packages/wordcloud/__init__.py)

In [12]:


p1 = figure(width=300, height=300, name='Scatter', margin=5)
p1.scatter([0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 3, 2, 1, 0])

p2 = figure(width=300, height=300, name='Line', margin=5)
p2.line([0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 3, 2, 1, 0])

accordion = pn.Accordion(('Scatter', p1), p2)
accordion


Accordion
    [0] Bokeh(Figure, name='Scatter')
    [1] Bokeh(Figure, name='Line')

In [14]:
accordion.print_param_values()

Accordion00115.active = [0, 1]
Accordion00115.active_header_background = #ccc
Accordion00115.align = start
Accordion00115.aspect_ratio = None
Accordion00115.background = None
Accordion00115.css_classes = None
Accordion00115.header_background = 
Accordion00115.header_color = 
Accordion00115.height = None
Accordion00115.height_policy = auto
Accordion00115.loading = False
Accordion00115.margin = 0
Accordion00115.max_height = None
Accordion00115.max_width = None
Accordion00115.min_height = None
Accordion00115.min_width = None
Accordion00115.name = Accordion00115
Accordion00115.objects = [Bokeh(Figure, name='Scatter'), Bokeh(Figure, name='Line')]
Accordion00115.scroll = False
Accordion00115.sizing_mode = None
Accordion00115.toggle = False
Accordion00115.visible = True
Accordion00115.width = None
Accordion00115.width_policy = auto


In [15]:
accordion.header_color = 'white'
accordion.header_background = 'blue'

In [16]:
accordion

Accordion(active=[0, 1], header_background='blue', header_color='white')
    [0] Bokeh(Figure, name='Scatter')
    [1] Bokeh(Figure, name='Line')